In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
import geopandas as gpd
from scipy import spatial

# SAFEGRAPH DATA ANALYSIS

In [2]:
filepath='/Users/naroacorettisanchez/Documents/GitHub/FoodDeliveries/preprocessing/Input_DataBase_Generation/Cambridge'

In [3]:
# Import patterns and places data from SafeGraph
filepath_safegraph_patterns = filepath + "/SafeGraph/Patterns/"
filepath_safegraph_places = filepath + "/SafeGraph/Places/"
safegraph_patterns_df = pd.read_csv(filepath_safegraph_patterns + 'your_data_may_26_2022_0138pm.csv')
safegraph_places_df = pd.read_csv(filepath_safegraph_places + 'your_data_may_26_2022_0452pm.csv')

In [4]:
# Select the columns of patterns and places that are of interest
patterns_df = safegraph_patterns_df[['placekey', 'raw_visit_counts', 'raw_visitor_counts', 'visits_by_day',
        'visits_by_each_hour', 'poi_cbg', 'visitor_home_cbgs', 'visitor_daytime_cbgs', 'distance_from_home', 
        'normalized_visits_by_state_scaling','date_range_start', 'date_range_end', 'median_dwell','bucketed_dwell_times' ]]
places_df=safegraph_places_df[['placekey','latitude','longitude','top_category','open_hours', 'location_name']]

In [5]:
# Merge patterns with places to have lat, lon, top category
merged_df=patterns_df.merge(places_df[['placekey','latitude','longitude', 'top_category', 'open_hours', 'location_name']], how='left', left_on='placekey', right_on='placekey')
merged_df.head()

,placekey,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,visitor_daytime_cbgs,distance_from_home,normalized_visits_by_state_scaling,date_range_start,date_range_end,median_dwell,bucketed_dwell_times,latitude,longitude,top_category,open_hours,location_name
0,223-222@62j-sj2-dvz,86,37,"[13,12,17,12,14,7,11]","[0,0,0,0,0,0,1,1,1,1,2,0,1,0,1,0,2,0,2,0,1,0,0...",250173531022,"{""371259506011"":4,""090091660024"":4,""1703141070...","{""250173331005"":4,""250173531022"":4,""3718305260...",37297.0,1392.490743,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,245.0,"{""<5"":0,""5-10"":8,""11-20"":4,""21-60"":9,""61-120"":...",42.362149,-71.087252,Traveler Accommodation,NaN,Kendall Hotel
1,22b-222@62j-sjv-nyv,12,10,"[1,1,2,1,2,1,4]","[0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",250173542003,"{""250173703003"":4}","{""250173324002"":4}",11918.0,194.301034,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,14.5,"{""<5"":0,""5-10"":6,""11-20"":1,""21-60"":0,""61-120"":...",42.378209,-71.130778,Religious Organizations,NaN,Holy Trinity Armenian Apostle Church Of Greate...
2,22g-222@62j-sk2-vmk,13,13,"[1,3,0,1,1,6,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0...",250173547001,"{""250250401001"":4,""250173547002"":4,""2501735070...","{""250173507005"":4,""250251701007"":4}",4306.0,210.492787,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,20.0,"{""<5"":0,""5-10"":5,""11-20"":2,""21-60"":5,""61-120"":...",42.388898,-71.118102,"Sporting Goods, Hobby, and Musical Instrument ...","{ ""Mon"": [], ""Tue"": [[""11:00"", ""18:00""]], ""Wed...",Mind's Eye Yarns
3,zzy-222@62j-sjr-zvf,24,8,"[3,5,5,3,8,0,0]","[0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0...",250173544001,"{""250056532032"":4,""250092103003"":4,""4202701060...","{""250173544001"":4,""250138104142"":4}",69898.0,388.602068,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,53.0,"{""<5"":0,""5-10"":2,""11-20"":4,""21-60"":8,""61-120"":...",42.386489,-71.140751,Other Amusement and Recreation Industries,NaN,Callanan Playground
4,zzz-222@62j-sj2-sbk,6368,4476,"[338,553,1072,1550,1069,924,862]","[13,10,15,10,17,15,19,10,24,9,16,16,15,11,17,1...",250173531012,"{""250173531012"":156,""360470507001"":94,""2612516...","{""250173531012"":226,""360470507001"":83,""2502598...",125161.0,103109.082016,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,251.0,"{""<5"":25,""5-10"":238,""11-20"":231,""21-60"":603,""6...",42.364692,-71.098640,"Museums, Historical Sites, and Similar Institu...",NaN,Columbia Park


In [6]:
#Keep points of interest related to food
topcategory_list =['Drinking Places (Alcoholic Beverages)', 'Restaurants and Other Eating Places', 'Specialty Food Services']
filtered_df = merged_df[merged_df['top_category'].isin(topcategory_list)]
filtered_df=filtered_df.reset_index().drop('index', axis =1)
filtered_df.head()

,placekey,raw_visit_counts,raw_visitor_counts,visits_by_day,visits_by_each_hour,poi_cbg,visitor_home_cbgs,visitor_daytime_cbgs,distance_from_home,normalized_visits_by_state_scaling,date_range_start,date_range_end,median_dwell,bucketed_dwell_times,latitude,longitude,top_category,open_hours,location_name
0,237-222@62j-sk2-vs5,3,2,"[0,0,0,0,0,2,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",250173547001,"{""250173535002"":4}",{},NaN,48.575258,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,146.0,"{""<5"":0,""5-10"":0,""11-20"":1,""21-60"":0,""61-120"":...",42.388759,-71.119907,Restaurants and Other Eating Places,NaN,Passage to India
1,23g-222@62j-sjv-9zz,11,11,"[1,1,2,0,1,5,1]","[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0...",250173539003,"{""250235012012"":4,""250250806011"":4}","{""250039002006"":4,""250173422012"":4,""0605905242...",174005.0,178.109281,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,53.0,"{""<5"":0,""5-10"":2,""11-20"":1,""21-60"":4,""61-120"":...",42.371790,-71.120643,Restaurants and Other Eating Places,NaN,Shays Pub & Wine Bar
2,22d-222@62j-sjr-zcq,35,30,"[7,3,8,5,2,5,5]","[0,0,0,0,0,0,2,1,0,1,1,0,0,0,0,0,0,2,0,0,0,0,0...",250173546004,"{""360530309001"":4,""250173571002"":4,""4500701010...","{""250173546004"":4,""250173566011"":4,""2501735780...",4440.0,566.711349,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,7.0,"{""<5"":1,""5-10"":28,""11-20"":3,""21-60"":1,""61-120""...",42.387839,-71.142286,Restaurants and Other Eating Places,"{ ""Mon"": [[""4:00"", ""24:00""]], ""Tue"": [[""0:00"",...",Dunkin'
3,22j-222@62j-sj2-rkz,14,14,"[3,2,0,7,1,1,0]","[0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0...",250173530002,"{""250173162021"":4,""250173398014"":4,""2501735300...","{""250173398014"":4}",7013.0,226.684540,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,8.0,"{""<5"":1,""5-10"":7,""11-20"":4,""21-60"":1,""61-120"":...",42.365675,-71.103734,Restaurants and Other Eating Places,"{ ""Mon"": [[""5:30"", ""20:30""]], ""Tue"": [[""5:30"",...",Starbucks
4,239-222@62j-sj2-7dv,64,34,"[11,14,12,9,6,8,4]","[0,0,0,1,0,0,0,1,3,3,0,0,2,1,0,0,0,0,0,0,0,0,0...",250173526001,"{""250277151001"":4,""250173531021"":4,""2501733850...","{""250056161002"":4,""250173872011"":4,""2501736520...",18133.0,1036.272181,2022-05-16T00:00:00-04:00,2022-05-23T00:00:00-04:00,163.5,"{""<5"":0,""5-10"":10,""11-20"":5,""21-60"":5,""61-120""...",42.367958,-71.090294,Restaurants and Other Eating Places,NaN,Earnest Drinks


In [7]:
# From filtered file -> Change visits by hour (1 column) to actual columns (24: one by hour)
hour_df = pd.DataFrame()
hour_df ['placekey'] = filtered_df ['placekey'].copy()
hour_df ['poi_cbg'] = filtered_df['poi_cbg'].copy()
for i in range(len(filtered_df)):
    aux = json.loads(filtered_df['visits_by_each_hour'][i])
    for j in range(24):
        hour_df.loc[i, j] = aux[j]
hour_df.head()

,placekey,poi_cbg,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,237-222@62j-sk2-vs5,250173547001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23g-222@62j-sjv-9zz,250173539003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,22d-222@62j-sjr-zcq,250173546004,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,22j-222@62j-sj2-rkz,250173530002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,239-222@62j-sj2-7dv,250173526001,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Change the type of numeric values and sort data by blockgroup
for i in range(24):
    hour_df[i] = hour_df[i].astype('Int64')

hour_df = hour_df.sort_values(by='poi_cbg', ascending = True).reset_index().drop('index', axis =1)
hour_df.head()

,placekey,poi_cbg,0,1,2,3,4,5,6,7,...,14,15,16,17,18,19,20,21,22,23
0,225-222@62j-sj2-ygk,250173515001,0,0,1,0,0,0,1,1,...,4,7,2,2,1,2,1,2,4,0
1,zzw-223@62j-shx-gx5,250173521011,0,0,0,0,0,1,1,3,...,0,2,0,1,0,0,0,0,0,0
2,zzw-222@62j-shx-gx5,250173521011,0,0,0,0,0,0,1,0,...,1,1,0,0,1,0,0,0,0,0
3,zzy-222@62j-shx-m8v,250173521012,0,0,0,0,0,0,0,0,...,2,0,0,0,0,2,1,0,0,1
4,226-222@62j-shx-68v,250173521012,0,0,0,0,0,0,2,0,...,0,1,0,0,0,0,0,0,0,0


In [9]:
# Group by blockgroup and sum the visits by blockgroup
hour_bg_groupped = hour_df.groupby('poi_cbg').sum()
hour_bg_groupped = hour_bg_groupped.reset_index()
hour_bg_groupped.head()

,poi_cbg,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,250173515001,0,0,1,0,0,0,1,1,1,...,4,7,2,2,1,2,1,2,4,0
1,250173521011,0,0,0,0,0,1,2,3,3,...,1,3,0,1,1,0,0,0,0,0
2,250173521012,0,0,0,0,0,0,2,0,3,...,2,1,0,0,0,2,1,0,0,1
3,250173521021,0,0,0,0,3,0,2,1,2,...,1,4,2,1,5,2,5,0,0,0
4,250173521022,0,0,0,0,0,0,1,0,1,...,1,0,2,2,1,0,0,0,0,0


In [10]:
# Calculate Probabilities
list_bg = hour_bg_groupped['poi_cbg'].unique().tolist()
aux_poi_cbg = pd.DataFrame()
for i in range(len(hour_df)):
    for j in range(24):
        bg = hour_bg_groupped[hour_bg_groupped['poi_cbg'] == hour_df.at[i, 'poi_cbg']]
        divider = bg[j].item()
        if divider != 0:
            hour_df.at[i, 'Prob_'+str(j)] = round(hour_df.loc[i,j]/divider,3)
        else:
            hour_df.at[i, 'Prob_'+str(j)] = 0
hour_df.head()

,placekey,poi_cbg,0,1,2,3,4,5,6,7,...,Prob_14,Prob_15,Prob_16,Prob_17,Prob_18,Prob_19,Prob_20,Prob_21,Prob_22,Prob_23
0,225-222@62j-sj2-ygk,250173515001,0,0,1,0,0,0,1,1,...,1.0,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
1,zzw-223@62j-shx-gx5,250173521011,0,0,0,0,0,1,1,3,...,0.0,0.667,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,zzw-222@62j-shx-gx5,250173521011,0,0,0,0,0,0,1,0,...,1.0,0.333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,zzy-222@62j-shx-m8v,250173521012,0,0,0,0,0,0,0,0,...,1.0,0.000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,226-222@62j-shx-68v,250173521012,0,0,0,0,0,0,2,0,...,0.0,1.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Merge Probabilities with places to have lat, lon
merge_hour_df=hour_df.merge(places_df[['placekey','latitude','longitude', 'top_category']], how='left', left_on='placekey', right_on='placekey')
merge_hour_df.head()

,placekey,poi_cbg,0,1,2,3,4,5,6,7,...,Prob_17,Prob_18,Prob_19,Prob_20,Prob_21,Prob_22,Prob_23,latitude,longitude,top_category
0,225-222@62j-sj2-ygk,250173515001,0,0,1,0,0,0,1,1,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,42.373710,-71.082864,Restaurants and Other Eating Places
1,zzw-223@62j-shx-gx5,250173521011,0,0,0,0,0,1,1,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,42.371960,-71.075408,Restaurants and Other Eating Places
2,zzw-222@62j-shx-gx5,250173521011,0,0,0,0,0,0,1,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,42.371938,-71.075402,Restaurants and Other Eating Places
3,zzy-222@62j-shx-m8v,250173521012,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,42.369810,-71.071383,Restaurants and Other Eating Places
4,226-222@62j-shx-68v,250173521012,0,0,0,0,0,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.369859,-71.068824,Restaurants and Other Eating Places


In [12]:
# Drop visits per hour and keep probabilities
drop_list=[ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,  22, 23]
merge_hour_df=merge_hour_df.drop(drop_list,axis=1)
merge_hour_df.head()

,placekey,poi_cbg,Prob_0,Prob_1,Prob_2,Prob_3,Prob_4,Prob_5,Prob_6,Prob_7,...,Prob_17,Prob_18,Prob_19,Prob_20,Prob_21,Prob_22,Prob_23,latitude,longitude,top_category
0,225-222@62j-sj2-ygk,250173515001,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,42.373710,-71.082864,Restaurants and Other Eating Places
1,zzw-223@62j-shx-gx5,250173521011,0.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,42.371960,-71.075408,Restaurants and Other Eating Places
2,zzw-222@62j-shx-gx5,250173521011,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,42.371938,-71.075402,Restaurants and Other Eating Places
3,zzy-222@62j-shx-m8v,250173521012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,42.369810,-71.071383,Restaurants and Other Eating Places
4,226-222@62j-shx-68v,250173521012,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.369859,-71.068824,Restaurants and Other Eating Places


# REPLICA DATA ANALYSIS

In [13]:
# INSERT REPLICA TRIPS DATA
filepath_Replica_Trips = filepath + '/Replica/Trips_BQ/'

#WEEKDAY
#replica_trips_df = pd.read_csv(filepath_Replica_Trips + 'replica_cambridge_bq_df.csv')

#WEEKEND
replica_trips_df = pd.read_csv(filepath_Replica_Trips + 'replica_cambridge_bq_df_sat.csv')
replica_trips_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,start_local_hour,end_time,...,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology
0,4.819264e+18,42.376375,-71.128129,42.372997,-71.078773,HOME,SOCIAL,09:41:58,9,09:50:53,...,single_family,residential,single_family,residential,multi_family,residential,multi_family,NaN,NaN,NaN
1,1.718374e+19,42.382151,-71.114950,42.381898,-71.114965,HOME,SOCIAL,11:59:21,11,11:59:39,...,single_family,residential,single_family,residential,multi_family,residential,multi_family,NaN,NaN,NaN
2,1.650718e+19,42.382384,-71.114577,42.386105,-71.124642,WORK,HOME,16:23:18,16,16:48:44,...,multi_family,residential,multi_family,mixed_use,education,civic_institutional,education,NaN,NaN,NaN
3,1.512663e+19,42.381358,-71.114231,42.371568,-71.120478,HOME,SOCIAL,17:05:50,17,17:10:15,...,single_family,residential,single_family,civic_institutional,education,civic_institutional,education,NaN,NaN,NaN
4,4.914142e+18,42.371433,-71.107481,42.385490,-71.119628,SHOP,HOME,11:36:56,11,11:42:01,...,multi_family,residential,multi_family,commercial,retail,commercial,retail,NaN,NaN,NaN


In [14]:
drop_list = ['start_local_hour', 'end_local_hour','origin_bgrp_lat', 'origin_bgrp_lng','destination_bgrp_lat', 'destination_bgrp_lng',
             'origin_building_use_l1', 'origin_building_use_l2', 'destination_building_use_l1','destination_building_use_l2']
replica_trips_df=replica_trips_df.drop(drop_list, axis=1)
replica_trips_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,origin_bgrp,destination_bgrp,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology
0,4.819264e+18,42.376375,-71.128129,42.372997,-71.078773,HOME,SOCIAL,09:41:58,09:50:53,8,3.068502,250173541002,250173521011,residential,single_family,residential,multi_family,NaN,NaN,NaN
1,1.718374e+19,42.382151,-71.114950,42.381898,-71.114965,HOME,SOCIAL,11:59:21,11:59:39,0,0.016212,250173536003,250173536003,residential,single_family,residential,multi_family,NaN,NaN,NaN
2,1.650718e+19,42.382384,-71.114577,42.386105,-71.124642,WORK,HOME,16:23:18,16:48:44,25,1.317140,250173536003,250173545002,residential,multi_family,mixed_use,education,NaN,NaN,NaN
3,1.512663e+19,42.381358,-71.114231,42.371568,-71.120478,HOME,SOCIAL,17:05:50,17:10:15,4,1.197344,250173536003,250173539003,residential,single_family,civic_institutional,education,NaN,NaN,NaN
4,4.914142e+18,42.371433,-71.107481,42.385490,-71.119628,SHOP,HOME,11:36:56,11:42:01,5,1.594008,250173538001,250173545001,residential,multi_family,commercial,retail,NaN,NaN,NaN


In [15]:
# Filter trips by trip purpose = EAT
replica_eat_df = replica_trips_df[replica_trips_df['travel_purpose'] == 'EAT']
replica_eat_df=replica_eat_df.reset_index().drop('index',axis=1)
replica_eat_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,distance_miles,origin_bgrp,destination_bgrp,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology
0,1.069494e+19,42.381172,-71.116043,42.364998,-71.102425,EAT,HOME,09:02:33,09:07:28,4,1.750200,250173536003,250173530002,residential,multi_family,commercial,retail,NaN,NaN,NaN
1,1.149327e+19,42.371376,-71.108998,42.373523,-71.119595,EAT,HOME,18:20:43,18:23:07,2,0.672914,250173538001,250173540003,residential,multi_family,commercial,retail,NaN,NaN,NaN
2,1.229746e+19,42.371328,-71.107260,42.371801,-71.120534,EAT,HOME,09:49:14,10:07:09,17,0.928116,250173538001,250173539003,residential,multi_family,commercial,retail,NaN,NaN,NaN
3,1.539994e+19,42.374856,-71.125801,42.372005,-71.120345,EAT,SOCIAL,10:07:58,10:15:45,7,0.403395,250173541002,250173539003,residential,multi_family,commercial,retail,NaN,NaN,NaN
4,5.632022e+18,42.373917,-71.125980,42.383544,-71.133720,EAT,HOME,12:31:09,12:34:44,3,1.144237,250173541002,250173544002,residential,single_family,commercial,retail,NaN,NaN,NaN


In [16]:
len(replica_eat_df)

14161

In [711]:
# # Destination landuse commercial
# replica_eat_df = replica_eat_df[replica_eat_df['destination_land_use_l1']=='commercial']
# replica_eat_df=replica_eat_df.reset_index().drop('index',axis=1)
# replica_eat_df.head()

In [712]:
len(replica_eat_df)

11156

In [17]:
# Change blockgroup data to int so that it matches blockgroup data in Safegraph files
replica_eat_df['destination_bgrp']=replica_eat_df['destination_bgrp'].astype(np.int64)
# Generate a new column with the start hour of the trips to make a match
replica_eat_df['start_hour'] = replica_eat_df['start_time'].str[:2].astype(np.int64)
replica_eat_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,origin_bgrp,destination_bgrp,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour
0,1.069494e+19,42.381172,-71.116043,42.364998,-71.102425,EAT,HOME,09:02:33,09:07:28,4,...,250173536003,250173530002,residential,multi_family,commercial,retail,NaN,NaN,NaN,9
1,1.149327e+19,42.371376,-71.108998,42.373523,-71.119595,EAT,HOME,18:20:43,18:23:07,2,...,250173538001,250173540003,residential,multi_family,commercial,retail,NaN,NaN,NaN,18
2,1.229746e+19,42.371328,-71.107260,42.371801,-71.120534,EAT,HOME,09:49:14,10:07:09,17,...,250173538001,250173539003,residential,multi_family,commercial,retail,NaN,NaN,NaN,9
3,1.539994e+19,42.374856,-71.125801,42.372005,-71.120345,EAT,SOCIAL,10:07:58,10:15:45,7,...,250173541002,250173539003,residential,multi_family,commercial,retail,NaN,NaN,NaN,10
4,5.632022e+18,42.373917,-71.125980,42.383544,-71.133720,EAT,HOME,12:31:09,12:34:44,3,...,250173541002,250173544002,residential,single_family,commercial,retail,NaN,NaN,NaN,12


In [18]:
# Assign a restaurant from blockgroup based on probabilities
from random import choices
len_trips = len(replica_eat_df)
trips_bg = pd.DataFrame(replica_eat_df['destination_bgrp'].values)
trips_bg = trips_bg.rename(columns={0: "destination_bgrp"}, errors="raise")
probabs = pd.DataFrame()
cho = np.empty(len_trips, dtype=object)
count=0
for i in range (len_trips):
    trip_bg = trips_bg['destination_bgrp'].values[i]
    restaurants = merge_hour_df[merge_hour_df['poi_cbg'] == trip_bg]
    
    probabs = restaurants[['placekey', 'Prob_' +str(replica_eat_df['start_hour'].values[i])]]
    aux=0
    for j in range(len(probabs)):
        if probabs['Prob_' +str(replica_eat_df['start_hour'].values[i])].values[j]!=0.0 :
            aux = 1
            break
        else:
            aux=0
    if aux==1:
        choice=choices(probabs['placekey'].to_list(), probabs ['Prob_' + str(replica_eat_df['start_hour'].values[i])].to_list())
        choice_aux = choice[0]
        cho[i]=choice_aux
    else:
        cho[i]='nothing'
        count=count+1

replica_eat_df['placekey']=cho
replica_eat_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_bgrp,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey
0,1.069494e+19,42.381172,-71.116043,42.364998,-71.102425,EAT,HOME,09:02:33,09:07:28,4,...,250173530002,residential,multi_family,commercial,retail,NaN,NaN,NaN,9,nothing
1,1.149327e+19,42.371376,-71.108998,42.373523,-71.119595,EAT,HOME,18:20:43,18:23:07,2,...,250173540003,residential,multi_family,commercial,retail,NaN,NaN,NaN,18,23k-223@62j-sjv-99f
2,1.229746e+19,42.371328,-71.107260,42.371801,-71.120534,EAT,HOME,09:49:14,10:07:09,17,...,250173539003,residential,multi_family,commercial,retail,NaN,NaN,NaN,9,zzw-222@62j-sjv-9vf
3,1.539994e+19,42.374856,-71.125801,42.372005,-71.120345,EAT,SOCIAL,10:07:58,10:15:45,7,...,250173539003,residential,multi_family,commercial,retail,NaN,NaN,NaN,10,225-222@62j-sjv-8sq
4,5.632022e+18,42.373917,-71.125980,42.383544,-71.133720,EAT,HOME,12:31:09,12:34:44,3,...,250173544002,residential,single_family,commercial,retail,NaN,NaN,NaN,12,nothing


# MERGE SAFEGRAPH AND REPLICA

In [19]:
# Add latitude and longitude of every assigned restaurant
eat_df = replica_eat_df.merge(filtered_df[['placekey','latitude', 'longitude']], how='left', left_on='placekey', right_on='placekey')
eat_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,latitude,longitude
0,1.069494e+19,42.381172,-71.116043,42.364998,-71.102425,EAT,HOME,09:02:33,09:07:28,4,...,multi_family,commercial,retail,NaN,NaN,NaN,9,nothing,NaN,NaN
1,1.149327e+19,42.371376,-71.108998,42.373523,-71.119595,EAT,HOME,18:20:43,18:23:07,2,...,multi_family,commercial,retail,NaN,NaN,NaN,18,23k-223@62j-sjv-99f,42.373493,-71.120642
2,1.229746e+19,42.371328,-71.107260,42.371801,-71.120534,EAT,HOME,09:49:14,10:07:09,17,...,multi_family,commercial,retail,NaN,NaN,NaN,9,zzw-222@62j-sjv-9vf,42.371807,-71.120206
3,1.539994e+19,42.374856,-71.125801,42.372005,-71.120345,EAT,SOCIAL,10:07:58,10:15:45,7,...,multi_family,commercial,retail,NaN,NaN,NaN,10,225-222@62j-sjv-8sq,42.371329,-71.116471
4,5.632022e+18,42.373917,-71.125980,42.383544,-71.133720,EAT,HOME,12:31:09,12:34:44,3,...,single_family,commercial,retail,NaN,NaN,NaN,12,nothing,NaN,NaN


# REASSIGNMENT

In [20]:
# Trips where the restaurants from the blockgroup have probability=0 to happen
nonassigned_df=eat_df[eat_df['placekey']=='nothing']
drop_list = ['placekey','latitude','longitude']
nonassigned_df=nonassigned_df.drop(drop_list, axis=1)
nonassigned_df=nonassigned_df.reset_index().drop('index',axis=1)
nonassigned_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,origin_bgrp,destination_bgrp,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour
0,1.069494e+19,42.381172,-71.116043,42.364998,-71.102425,EAT,HOME,09:02:33,09:07:28,4,...,250173536003,250173530002,residential,multi_family,commercial,retail,NaN,NaN,NaN,9
1,5.632022e+18,42.373917,-71.125980,42.383544,-71.133720,EAT,HOME,12:31:09,12:34:44,3,...,250173541002,250173544002,residential,single_family,commercial,retail,NaN,NaN,NaN,12
2,1.545698e+19,42.371149,-71.109150,42.365675,-71.103734,EAT,HOME,10:48:43,11:02:03,13,...,250173538001,250173530002,residential,multi_family,commercial,retail,NaN,NaN,NaN,10
3,1.024543e+19,42.370167,-71.105833,42.372277,-71.079815,EAT,HOME,09:30:53,09:35:55,5,...,250173538001,250173521022,residential,multi_family,commercial,retail,NaN,NaN,NaN,9
4,2.589153e+18,42.381414,-71.114617,42.369080,-71.100948,EAT,HOME,11:46:00,11:51:26,5,...,250173536003,250173530002,residential,single_family,commercial,retail,NaN,NaN,NaN,11


In [21]:
# Get the visits by day to the restaurants in the blockgroups with trips not assigned
blockgroups=nonassigned_df['destination_bgrp'].unique()
week_df = pd.DataFrame()
fweek_df = pd.DataFrame()
week_df ['placekey'] = filtered_df ['placekey'].copy()
week_df ['poi_cbg'] = filtered_df['poi_cbg'].copy()
aux_df=pd.DataFrame()
for i in range (len(blockgroups)):
    aux_df=week_df[week_df['poi_cbg']==blockgroups[i]]
    fweek_df=pd.concat([fweek_df,aux_df])
fweek_df=fweek_df.merge(filtered_df[['placekey', 'visits_by_day']], how='left', left_on='placekey', right_on='placekey')
fweek_df

,placekey,poi_cbg,visits_by_day
0,22j-222@62j-sj2-rkz,250173530002,"[3,2,0,7,1,1,0]"
1,23p-222@62j-sj2-5j9,250173530002,"[4,4,2,7,4,4,1]"
2,zzw-227@62j-sj2-p35,250173530002,"[0,0,1,0,3,0,1]"
3,zzw-225@62j-sj2-p35,250173530002,"[2,1,3,5,5,7,2]"
4,22b-222@62j-sj2-rff,250173530002,"[12,9,10,6,8,17,4]"
...,...,...,...
268,zzw-222@62j-sj4-rp9,250173531022,"[12,12,6,5,6,3,5]"
269,zzy-222@62j-sj4-v9f,250173531022,"[8,10,8,15,14,3,4]"
270,222-223@62j-sj2-dqf,250173531022,"[39,41,48,28,22,32,7]"
271,223-223@62j-sj2-dvz,250173531022,"[5,5,10,8,9,4,5]"


In [22]:
# Calculate the total visits per week for those restaurants
for i in range(len(fweek_df)):
    aux = json.loads(fweek_df['visits_by_day'][i])
    for j in range(7):
        fweek_df.loc[i, j] = aux[j]
for i in range(7):
    fweek_df[i] = fweek_df[i].astype('Int64')
fweek_df=fweek_df.rename(columns={0: "Monday"}, errors="raise")        
fweek_df=fweek_df.rename(columns={1: "Tuesday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={2: "Wednesday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={3: "Thrusday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={4: "Friday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={5: "Saturday"}, errors="raise") 
fweek_df=fweek_df.rename(columns={6: "Sunday"}, errors="raise") 
fweek_df['Total_Visits']=fweek_df['Monday']+fweek_df['Tuesday']+fweek_df['Wednesday']+fweek_df['Thrusday']+fweek_df['Friday']+fweek_df['Saturday']+fweek_df['Sunday']
fweek_df = fweek_df.sort_values(by='poi_cbg', ascending = True).reset_index().drop('index', axis =1)
fweek_df.head()

,placekey,poi_cbg,visits_by_day,Monday,Tuesday,Wednesday,Thrusday,Friday,Saturday,Sunday,Total_Visits
0,zzw-222@62j-shx-gx5,250173521011,"[10,7,5,9,13,6,3]",10,7,5,9,13,6,3,53
1,zzw-223@62j-shx-gx5,250173521011,"[19,20,31,20,16,9,3]",19,20,31,20,16,9,3,118
2,226-222@62j-shx-68v,250173521012,"[10,10,9,9,7,2,2]",10,10,9,9,7,2,2,49
3,zzy-222@62j-shx-m8v,250173521012,"[10,6,8,6,3,5,6]",10,6,8,6,3,5,6,44
4,zzw-222@62j-shx-td9,250173521021,"[17,14,10,10,15,23,9]",17,14,10,10,15,23,9,98


In [23]:
# Drop visits per hour and keep probabilities
drop_list=[ 'visits_by_day', 'Monday', 'Tuesday', 'Wednesday', 'Thrusday', 'Friday', 'Saturday', 'Sunday']
fweek_df=fweek_df.drop(drop_list,axis=1)
fweek_df=fweek_df.merge(filtered_df[['placekey', 'open_hours']], how='left', left_on='placekey', right_on='placekey')
fweek_df=fweek_df.dropna().reset_index().drop('index',axis=1)
fweek_df.head()

,placekey,poi_cbg,Total_Visits,open_hours
0,226-222@62j-shx-68v,250173521012,49,"{ ""Mon"": [[""8:00"", ""21:00""]], ""Tue"": [[""8:00"",..."
1,zzw-222@62j-shx-td9,250173521021,98,"{ ""Mon"": [[""6:30"", ""15:00""]], ""Tue"": [[""6:30"",..."
2,zzw-222@62j-shx-jvz,250173521021,28,"{ ""Mon"": [[""11:00"", ""22:00""]], ""Tue"": [[""11:00..."
3,zzw-222@62j-shx-kcq,250173521021,29,"{ ""Mon"": [[""11:00"", ""14:00""]], ""Tue"": [[""11:00..."
4,243-222@62j-shx-j7q,250173521022,22,"{ ""Mon"": [[""5:00"", ""20:00""]], ""Tue"": [[""5:00"",..."


In [24]:
# Security copy
fweek_df_copy = fweek_df.copy()
fweek_df_copy=fweek_df_copy.dropna().reset_index().drop('index',axis=1)
fweek_df_copy.head()

,placekey,poi_cbg,Total_Visits,open_hours
0,226-222@62j-shx-68v,250173521012,49,"{ ""Mon"": [[""8:00"", ""21:00""]], ""Tue"": [[""8:00"",..."
1,zzw-222@62j-shx-td9,250173521021,98,"{ ""Mon"": [[""6:30"", ""15:00""]], ""Tue"": [[""6:30"",..."
2,zzw-222@62j-shx-jvz,250173521021,28,"{ ""Mon"": [[""11:00"", ""22:00""]], ""Tue"": [[""11:00..."
3,zzw-222@62j-shx-kcq,250173521021,29,"{ ""Mon"": [[""11:00"", ""14:00""]], ""Tue"": [[""11:00..."
4,243-222@62j-shx-j7q,250173521022,22,"{ ""Mon"": [[""5:00"", ""20:00""]], ""Tue"": [[""5:00"",..."


In [25]:
# Extracted open and close hours of each restaurant
import json
a = pd.DataFrame()
x = pd.DataFrame()
fweek_df_copy['Open']=' '
fweek_df_copy['Close']=' '
for i in range (len(fweek_df_copy)):
    a = json.loads(fweek_df_copy['open_hours'][i])
    x = a.get('Tue')
    if len(x)==0:
        x = a.get('Mon')
        if len(x)==0:
            x = a.get('Wed')
            if len(x)==0:
                x = a.get('Thu')
                if len(x)==0:
                    x = a.get('Fri')
                else:
                    fweek_df_copy.at[i,'Open']=x[0][0]
                    fweek_df_copy.at[i,'Close']=x[0][1]
            else:
                fweek_df_copy.at[i,'Open']=x[0][0]
                fweek_df_copy.at[i,'Close']=x[0][1]
        else:
            fweek_df_copy.at[i,'Open']=x[0][0]
            fweek_df_copy.at[i,'Close']=x[0][1]
    else:
        if x[0][1]=='24:00':
            x[0][1]='0:00'
            fweek_df_copy.at[i,'Close']=x[0][1]
        else:
            fweek_df_copy.at[i,'Close']=x[0][1]
        if x[0][0]=='0:00':
            x = a.get('Mon')
            if x[0][0]=='0:00':
                size=len(x)
                if size==1:
                    fweek_df_copy.at[i,'Open']=x[0][0]
                elif size==2:
                    fweek_df_copy.at[i,'Open']=x[1][0]
            else:
                fweek_df_copy.at[i,'Open']=x[0][0]
        else:
            fweek_df_copy.at[i,'Open']=x[0][0]
drop_list=['open_hours']
fweek_df_copy=fweek_df_copy.drop(drop_list, axis=1)
fweek_df_copy.head()

,placekey,poi_cbg,Total_Visits,Open,Close
0,226-222@62j-shx-68v,250173521012,49,8:00,21:00
1,zzw-222@62j-shx-td9,250173521021,98,6:30,15:00
2,zzw-222@62j-shx-jvz,250173521021,28,11:00,22:00
3,zzw-222@62j-shx-kcq,250173521021,29,11:00,14:00
4,243-222@62j-shx-j7q,250173521022,22,5:00,20:00


In [26]:
# Replace : by . to change type of variable
fweek_df_copy['Open'] = fweek_df_copy['Open'].replace({':' : '.'}, regex=True)
fweek_df_copy['Close'] = fweek_df_copy['Close'].replace({':' : '.'}, regex=True)
fweek_df_copy.head()

,placekey,poi_cbg,Total_Visits,Open,Close
0,226-222@62j-shx-68v,250173521012,49,8.00,21.00
1,zzw-222@62j-shx-td9,250173521021,98,6.30,15.00
2,zzw-222@62j-shx-jvz,250173521021,28,11.00,22.00
3,zzw-222@62j-shx-kcq,250173521021,29,11.00,14.00
4,243-222@62j-shx-j7q,250173521022,22,5.00,20.00


In [27]:
# Change open and close variables for a numeric one instead of a string
fweek_df_copy['Open'] = pd.to_numeric(fweek_df_copy['Open'],errors='coerce')
fweek_df_copy['Close'] = pd.to_numeric(fweek_df_copy['Close'],errors='coerce')
fweek_df_copy.head()

,placekey,poi_cbg,Total_Visits,Open,Close
0,226-222@62j-shx-68v,250173521012,49,8.0,21.0
1,zzw-222@62j-shx-td9,250173521021,98,6.3,15.0
2,zzw-222@62j-shx-jvz,250173521021,28,11.0,22.0
3,zzw-222@62j-shx-kcq,250173521021,29,11.0,14.0
4,243-222@62j-shx-j7q,250173521022,22,5.0,20.0


In [28]:
# Calculate visits per hour based on open hours and total visits per week
week_hours_df=fweek_df_copy.copy()
length=len(week_hours_df)
for i in range(24):
    week_hours_df[i]=' '
for i in range(length):
    hours_open = 0
    for j in range (24):
        if week_hours_df.at[i,'Close']<week_hours_df.at[i,'Open']:
            hours_open = 24 - abs (week_hours_df.at[i,'Close']-week_hours_df.at[i,'Open'])
            if j>=week_hours_df.at[i,'Open']:
                    week_hours_df.at[i,j]=(week_hours_df.at[i,'Total_Visits']/hours_open)
            else:
                if j>=week_hours_df.at[i,'Close']:
                    week_hours_df.at[i,j]=0
                else:
                    week_hours_df.at[i,j]=0
        else:
            if j>=week_hours_df.at[i,'Open']:
                hours_open = week_hours_df.at[i,'Close']-week_hours_df.at[i,'Open']
                if j<fweek_df_copy['Close'][i]:
                    week_hours_df.at[i,j]=(week_hours_df.at[i,'Total_Visits']/hours_open)
                else:
                    week_hours_df.at[i,j]=0
            else:
                week_hours_df.at[i,j]=0
    if week_hours_df.at[i,'Close']<week_hours_df.at[i,'Open']:
            aux=week_hours_df.at[i,'Close'].astype(int)
            for k in range(aux):
                week_hours_df.at[i,k]=(week_hours_df.at[i,'Total_Visits']/hours_open)
    
week_hours_df.head()

,placekey,poi_cbg,Total_Visits,Open,Close,0,1,2,3,4,...,14,15,16,17,18,19,20,21,22,23
0,226-222@62j-shx-68v,250173521012,49,8.0,21.0,0,0,0,0,0,...,3.76923,3.76923,3.76923,3.76923,3.76923,3.76923,3.76923,0,0,0
1,zzw-222@62j-shx-td9,250173521021,98,6.3,15.0,0,0,0,0,0,...,11.2644,0,0,0,0,0,0,0,0,0
2,zzw-222@62j-shx-jvz,250173521021,28,11.0,22.0,0,0,0,0,0,...,2.54545,2.54545,2.54545,2.54545,2.54545,2.54545,2.54545,2.54545,0,0
3,zzw-222@62j-shx-kcq,250173521021,29,11.0,14.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,243-222@62j-shx-j7q,250173521022,22,5.0,20.0,0,0,0,0,0,...,1.46667,1.46667,1.46667,1.46667,1.46667,1.46667,0,0,0,0


In [29]:
# Copy
wh_df=week_hours_df.copy()
wh_df.head()

,placekey,poi_cbg,Total_Visits,Open,Close,0,1,2,3,4,...,14,15,16,17,18,19,20,21,22,23
0,226-222@62j-shx-68v,250173521012,49,8.0,21.0,0,0,0,0,0,...,3.76923,3.76923,3.76923,3.76923,3.76923,3.76923,3.76923,0,0,0
1,zzw-222@62j-shx-td9,250173521021,98,6.3,15.0,0,0,0,0,0,...,11.2644,0,0,0,0,0,0,0,0,0
2,zzw-222@62j-shx-jvz,250173521021,28,11.0,22.0,0,0,0,0,0,...,2.54545,2.54545,2.54545,2.54545,2.54545,2.54545,2.54545,2.54545,0,0
3,zzw-222@62j-shx-kcq,250173521021,29,11.0,14.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,243-222@62j-shx-j7q,250173521022,22,5.0,20.0,0,0,0,0,0,...,1.46667,1.46667,1.46667,1.46667,1.46667,1.46667,0,0,0,0


In [30]:
# Change visits per hour from object to float
for i in range(24):
    wh_df[i]=wh_df[i].astype('Float64')

In [31]:
# Group by blockgroup and get the total weekly visits per week
week_bg_groupped = wh_df.groupby(wh_df['poi_cbg']).sum()
week_bg_groupped = week_bg_groupped.reset_index()
week_bg_groupped.head()

,poi_cbg,Total_Visits,Open,Close,0,1,2,3,4,5,...,14,15,16,17,18,19,20,21,22,23
0,250173521012,49,8.0,21.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,3.769231,3.769231,3.769231,3.769231,3.769231,3.769231,3.769231,0.000000,0.000000,0.0
1,250173521021,155,28.3,51.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,13.809822,2.545455,2.545455,2.545455,2.545455,2.545455,2.545455,2.545455,0.000000,0.0
2,250173521022,69,36.0,85.3,0.0,0.0,0.0,0.0,0.0,1.466667,...,5.286352,5.286352,5.286352,5.286352,5.286352,5.286352,3.819686,3.282051,2.615385,0.0
3,250173522001,114,81.0,141.3,1.2,1.2,0.0,0.0,0.0,0.000000,...,10.204391,10.204391,10.204391,10.204391,7.504391,6.129391,6.129391,6.129391,2.200000,1.2
4,250173523001,270,17.0,36.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,29.877778,1.100000,1.100000,1.100000,1.100000,1.100000,1.100000,0.000000,0.000000,0.0


In [32]:
# Calculate Probabilities based on week visits
for i in range(len(wh_df)):
    for j in range(24):
        bg_week = week_bg_groupped[week_bg_groupped['poi_cbg'] == wh_df.at[i, 'poi_cbg']]
        divider = bg_week[j].item()
        if divider != 0:
            wh_df.at[i, 'Prob_'+str(j)] = round(wh_df.loc[i,j]/divider,3)
        else:
            wh_df.at[i, 'Prob_'+str(j)] = 0
wh_df.head()

,placekey,poi_cbg,Total_Visits,Open,Close,0,1,2,3,4,...,Prob_14,Prob_15,Prob_16,Prob_17,Prob_18,Prob_19,Prob_20,Prob_21,Prob_22,Prob_23
0,226-222@62j-shx-68v,250173521012,49,8.0,21.0,0.0,0.0,0.0,0.0,0.0,...,1.000,1.000,1.000,1.000,1.000,1.000,1.0,0.0,0.0,0.0
1,zzw-222@62j-shx-td9,250173521021,98,6.3,15.0,0.0,0.0,0.0,0.0,0.0,...,0.816,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0
2,zzw-222@62j-shx-jvz,250173521021,28,11.0,22.0,0.0,0.0,0.0,0.0,0.0,...,0.184,1.000,1.000,1.000,1.000,1.000,1.0,1.0,0.0,0.0
3,zzw-222@62j-shx-kcq,250173521021,29,11.0,14.0,0.0,0.0,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0
4,243-222@62j-shx-j7q,250173521022,22,5.0,20.0,0.0,0.0,0.0,0.0,0.0,...,0.277,0.277,0.277,0.277,0.277,0.277,0.0,0.0,0.0,0.0


In [33]:
# Assign a restaurant from blockgroup based on probabilities
from random import choices
len_trips_na = len(nonassigned_df)
trips_bg_na = pd.DataFrame(nonassigned_df['destination_bgrp'].values)
trips_bg_na = trips_bg_na.rename(columns={0: "destination_bgrp"}, errors="raise")
probabs_na = pd.DataFrame()
cho_na = np.empty(len_trips_na, dtype=object)
count=0
for i in range (len_trips_na):
    trip_bg_na = trips_bg_na['destination_bgrp'].values[i]
    restaurants_na = wh_df[wh_df['poi_cbg'] == trip_bg_na]
    probabs_na = restaurants_na[['placekey', 'Prob_' +str(nonassigned_df['start_hour'].values[i])]]
    aux=0
    for j in range(len(probabs_na)):
        if probabs_na['Prob_' +str(nonassigned_df['start_hour'].values[i])].values[j]!=0.0 :
            aux = 1
            break
        else:
            aux=0
    if aux==1:
        choice_na=choices(probabs_na['placekey'].to_list(), probabs_na['Prob_' + str(nonassigned_df['start_hour'].values[i])].to_list())
        choice_aux = choice_na[0]
        cho_na[i]=choice_aux
    else:
        cho_na[i]='nothing'
        count=count+1

nonassigned_df['placekey']=cho_na
nonassigned_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_bgrp,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey
0,1.069494e+19,42.381172,-71.116043,42.364998,-71.102425,EAT,HOME,09:02:33,09:07:28,4,...,250173530002,residential,multi_family,commercial,retail,NaN,NaN,NaN,9,zzw-222@62j-sjp-jgk
1,5.632022e+18,42.373917,-71.125980,42.383544,-71.133720,EAT,HOME,12:31:09,12:34:44,3,...,250173544002,residential,single_family,commercial,retail,NaN,NaN,NaN,12,23m-222@62j-sjs-zpv
2,1.545698e+19,42.371149,-71.109150,42.365675,-71.103734,EAT,HOME,10:48:43,11:02:03,13,...,250173530002,residential,multi_family,commercial,retail,NaN,NaN,NaN,10,zzw-222@62j-sjp-jgk
3,1.024543e+19,42.370167,-71.105833,42.372277,-71.079815,EAT,HOME,09:30:53,09:35:55,5,...,250173521022,residential,multi_family,commercial,retail,NaN,NaN,NaN,9,243-222@62j-shx-j7q
4,2.589153e+18,42.381414,-71.114617,42.369080,-71.100948,EAT,HOME,11:46:00,11:51:26,5,...,250173530002,residential,single_family,commercial,retail,NaN,NaN,NaN,11,zzw-222@62j-sjp-jgk


In [34]:
l_non_assigned=len(nonassigned_df)
l_nothing=len(nonassigned_df[nonassigned_df['placekey']=='nothing'])

In [35]:
l_nothing

1053

In [36]:
dropped_trips=nonassigned_df[nonassigned_df['placekey']=='nothing'].reset_index().drop('index',axis=1)
dropped_trips

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_bgrp,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey
0,1.570032e+19,42.378229,-71.107649,42.368695,-71.094997,EAT,HOME,17:30:47,17:35:53,5,...,250173525001,residential,multi_family,mixed_use,multi_family,NaN,NaN,NaN,17,nothing
1,5.771527e+18,42.370420,-71.107362,42.382170,-71.130856,EAT,HOME,07:46:28,07:50:51,4,...,250173542003,residential,multi_family,commercial,retail,NaN,NaN,NaN,7,nothing
2,5.671159e+18,42.379122,-71.124818,42.391246,-71.123475,EAT,HOME,09:57:43,10:01:23,3,...,250173548002,residential,single_family,commercial,retail,NaN,NaN,NaN,9,nothing
3,4.343980e+18,42.382384,-71.114577,42.383171,-71.115991,EAT,HOME,00:38:45,00:41:12,2,...,250173536002,residential,multi_family,commercial,retail,NaN,NaN,NaN,0,nothing
4,1.436572e+19,42.377773,-71.112119,42.368400,-71.116001,EAT,HOME,16:35:25,16:53:55,18,...,250173539001,residential,single_family,civic_institutional,education,NaN,NaN,NaN,16,nothing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,1.369267e+17,42.382737,-71.120029,42.382011,-71.123983,EAT,WORK,18:43:46,18:44:54,1,...,250173540002,commercial,retail,mixed_use,education,NaN,NaN,NaN,18,nothing
1049,4.891730e+18,42.384144,-71.122818,42.382011,-71.123983,EAT,WORK_FROM_HOME,19:17:26,19:18:28,1,...,250173540002,residential,multi_family,mixed_use,education,NaN,NaN,NaN,19,nothing
1050,9.147914e+18,42.382404,-71.119990,42.382112,-71.119476,EAT,EAT,12:12:14,12:13:39,1,...,250173536002,commercial,retail,commercial,retail,NaN,NaN,NaN,12,nothing
1051,1.145765e+19,42.383351,-71.123273,42.382011,-71.123983,EAT,HOME,15:46:53,15:52:36,5,...,250173540002,residential,multi_family,mixed_use,education,NaN,NaN,NaN,15,nothing


In [37]:
# There is one trip that is done when all restaurants are closed so ignore
nonassigned_df.drop(nonassigned_df.loc[nonassigned_df['placekey']=='nothing'].index, inplace=True)
nonassigned_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_bgrp,origin_land_use_l1,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey
0,1.069494e+19,42.381172,-71.116043,42.364998,-71.102425,EAT,HOME,09:02:33,09:07:28,4,...,250173530002,residential,multi_family,commercial,retail,NaN,NaN,NaN,9,zzw-222@62j-sjp-jgk
1,5.632022e+18,42.373917,-71.125980,42.383544,-71.133720,EAT,HOME,12:31:09,12:34:44,3,...,250173544002,residential,single_family,commercial,retail,NaN,NaN,NaN,12,23m-222@62j-sjs-zpv
2,1.545698e+19,42.371149,-71.109150,42.365675,-71.103734,EAT,HOME,10:48:43,11:02:03,13,...,250173530002,residential,multi_family,commercial,retail,NaN,NaN,NaN,10,zzw-222@62j-sjp-jgk
3,1.024543e+19,42.370167,-71.105833,42.372277,-71.079815,EAT,HOME,09:30:53,09:35:55,5,...,250173521022,residential,multi_family,commercial,retail,NaN,NaN,NaN,9,243-222@62j-shx-j7q
4,2.589153e+18,42.381414,-71.114617,42.369080,-71.100948,EAT,HOME,11:46:00,11:51:26,5,...,250173530002,residential,single_family,commercial,retail,NaN,NaN,NaN,11,zzw-222@62j-sjp-jgk


In [38]:
# Add latitude and longitude of every assigned restaurant
na_df = nonassigned_df.merge(filtered_df[['placekey','latitude', 'longitude']], how='left', left_on='placekey', right_on='placekey')
na_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,latitude,longitude
0,1.069494e+19,42.381172,-71.116043,42.364998,-71.102425,EAT,HOME,09:02:33,09:07:28,4,...,multi_family,commercial,retail,NaN,NaN,NaN,9,zzw-222@62j-sjp-jgk,42.365138,-71.102738
1,5.632022e+18,42.373917,-71.125980,42.383544,-71.133720,EAT,HOME,12:31:09,12:34:44,3,...,single_family,commercial,retail,NaN,NaN,NaN,12,23m-222@62j-sjs-zpv,42.382482,-71.131438
2,1.545698e+19,42.371149,-71.109150,42.365675,-71.103734,EAT,HOME,10:48:43,11:02:03,13,...,multi_family,commercial,retail,NaN,NaN,NaN,10,zzw-222@62j-sjp-jgk,42.365138,-71.102738
3,1.024543e+19,42.370167,-71.105833,42.372277,-71.079815,EAT,HOME,09:30:53,09:35:55,5,...,multi_family,commercial,retail,NaN,NaN,NaN,9,243-222@62j-shx-j7q,42.372246,-71.079820
4,2.589153e+18,42.381414,-71.114617,42.369080,-71.100948,EAT,HOME,11:46:00,11:51:26,5,...,single_family,commercial,retail,NaN,NaN,NaN,11,zzw-222@62j-sjp-jgk,42.365138,-71.102738


# MERGE TRIPS

In [39]:
# Merge the assigned with the non assigned
d_list=['nothing']
eat_df=eat_df[eat_df['placekey'].isin(d_list)==False]
eattrips_df=pd.concat([eat_df,na_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
eattrips_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,latitude,longitude
0,7.119784e+18,42.371634,-71.120630,42.371734,-71.120416,EAT,HOME,00:47:05,00:47:33,0,...,education,commercial,retail,NaN,NaN,NaN,0,223-222@62j-sjv-9xq,42.370282,-71.118519
1,1.141718e+19,42.371590,-71.117231,42.371330,-71.116470,EAT,HOME,00:59:42,01:00:38,0,...,education,commercial,retail,NaN,NaN,NaN,0,223-222@62j-sjv-9xq,42.370282,-71.118519
2,1.367431e+19,42.373551,-71.120249,42.372097,-71.086866,EAT,EAT,01:14:48,01:19:51,5,...,office,mixed_use,retail,NaN,NaN,NaN,1,285-222@62j-sj2-x89,42.372127,-71.086885
3,1.761928e+18,42.392195,-71.123773,42.391419,-71.122663,EAT,HOME,02:31:22,02:33:59,2,...,multi_family,commercial,office,NaN,NaN,NaN,2,224-222@62j-sk2-tqf,42.389247,-71.120337
4,7.291174e+18,42.376795,-71.109011,42.372917,-71.119156,EAT,HOME,02:39:53,02:45:37,5,...,multi_family,commercial,retail,NaN,NaN,NaN,2,227-224@62j-sjv-b6k,42.372484,-71.117852


In [40]:
eattrips_df=eattrips_df.rename(columns={"latitude": "start_latitude"}, errors="raise")
eattrips_df=eattrips_df.rename(columns={"longitude": "start_longitude"}, errors="raise")
eattrips_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,7.119784e+18,42.371634,-71.120630,42.371734,-71.120416,EAT,HOME,00:47:05,00:47:33,0,...,education,commercial,retail,NaN,NaN,NaN,0,223-222@62j-sjv-9xq,42.370282,-71.118519
1,1.141718e+19,42.371590,-71.117231,42.371330,-71.116470,EAT,HOME,00:59:42,01:00:38,0,...,education,commercial,retail,NaN,NaN,NaN,0,223-222@62j-sjv-9xq,42.370282,-71.118519
2,1.367431e+19,42.373551,-71.120249,42.372097,-71.086866,EAT,EAT,01:14:48,01:19:51,5,...,office,mixed_use,retail,NaN,NaN,NaN,1,285-222@62j-sj2-x89,42.372127,-71.086885
3,1.761928e+18,42.392195,-71.123773,42.391419,-71.122663,EAT,HOME,02:31:22,02:33:59,2,...,multi_family,commercial,office,NaN,NaN,NaN,2,224-222@62j-sk2-tqf,42.389247,-71.120337
4,7.291174e+18,42.376795,-71.109011,42.372917,-71.119156,EAT,HOME,02:39:53,02:45:37,5,...,multi_family,commercial,retail,NaN,NaN,NaN,2,227-224@62j-sjv-b6k,42.372484,-71.117852


In [41]:
len(eattrips_df)

13108

# OSM DATA

In [42]:
# Import CAMBRIDGE Open Street Map Data to be able to assign exact location to the origin of the trips: 
# Transportation Utilities, Civic Institutional Buildings
filepath_osm_cambridge = filepath+"/OSM/"
osm_amenities_df = pd.read_csv(filepath_osm_cambridge + 'cambridgeamenitypois.csv')
osm_buildings_df = pd.read_csv(filepath_osm_cambridge + 'cambridgebuildingpois.csv')

# TRIP ORIGIN STUDY

In [43]:
eattrips_df_copy=eattrips_df.copy()

## CIVIC INSTITUTIONAL 

In [44]:
# Filter all the CIVIC INSTITUTIONAL buildings (based on origin land use 1)
replica_ci_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='civic_institutional'].reset_index().drop('index',axis=1)

In [45]:
# Drop CIVIC INSTITUTIONAL trips to check all of them assigned at the end
drop_list=['civic_institutional']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [46]:
# Split all CIVIC INSTITUTIONAL trips by the existing origin land use 2 values
replica_education_ci_df=replica_ci_df[replica_ci_df['origin_land_use_l2']=='education'].reset_index().drop('index',axis=1)
replica_healthcare_ci_df=replica_ci_df[replica_ci_df['origin_land_use_l2']=='healthcare'].reset_index().drop('index',axis=1)
replica_civic_institutional_ci_df=replica_ci_df[replica_ci_df['origin_land_use_l2']=='civic_institutional'].reset_index().drop('index',axis=1)

## COMMERCIAL 

In [47]:
# Filter all the COMMERCIAL buildings (based on origin land use 1)
replica_commercial_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='commercial'].reset_index().drop('index',axis=1)

In [48]:
# Drop COMMERCIAL trips to check all of them assigned at the end
drop_list=['commercial']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [49]:
# Split all COMMERCIAL trips by the existing origin land use 2 values
replica_office_c_df=replica_commercial_df[replica_commercial_df['origin_land_use_l2']=='office'].reset_index().drop('index',axis=1)
replica_retail_c_df=replica_commercial_df[replica_commercial_df['origin_land_use_l2']=='retail'].reset_index().drop('index',axis=1)
replica_non_retail_c_df=replica_commercial_df[replica_commercial_df['origin_land_use_l2']=='non_retail_attraction'].reset_index().drop('index',axis=1)

In [50]:
# Split all origin land use 2 = retail by previous activity type
restaurants_c_list=['EAT', 'SOCIAL','OTHER_ACTIVITY_TYPE']
retail_restaurants_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type'].isin(restaurants_c_list)].reset_index().drop('index',axis=1)
retail_shopping_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type']=='SHOP'].reset_index().drop('index',axis=1)
work_c_list=['WORK','MAINTENANCE']
retail_work_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type'].isin(work_c_list)].reset_index().drop('index',axis=1)
retail_park_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type']=='RECREATION'].reset_index().drop('index',axis=1)
retail_education_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type']=='SCHOOL'].reset_index().drop('index',axis=1)
retail_hotel_c_df=replica_retail_c_df[replica_retail_c_df['previous_activity_type']=='LODGING'].reset_index().drop('index',axis=1)

## MIXED USE

In [51]:
# Filter all the MIXED USE buildings (based on origin land use 1)
replica_mixed_use_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='mixed_use'].reset_index().drop('index',axis=1)

In [52]:
# Drop MIXED USE trips to check all of them assigned at the end
drop_list=['mixed_use']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [53]:
# Split all MIXED USE trips by the existing origin land use 2 values
replica_industrial_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='industrial'].reset_index().drop('index',axis=1)
replica_retail_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='retail'].reset_index().drop('index',axis=1)
replica_non_retail_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='non_retail_attraction'].reset_index().drop('index',axis=1)
replica_civic_institutional_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='civic_institutional'].reset_index().drop('index',axis=1)
residential_list=['multi_family','single_family']
replica_residential_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2'].isin(residential_list)].reset_index().drop('index',axis=1)
replica_education_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='education'].reset_index().drop('index',axis=1)
replica_office_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='office'].reset_index().drop('index',axis=1)
replica_healthcare_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='healthcare'].reset_index().drop('index',axis=1)
replica_park_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='open_space'].reset_index().drop('index',axis=1)
replica_tu_mu_df=replica_mixed_use_df[replica_mixed_use_df['origin_land_use_l2']=='transportation_utilities'].reset_index().drop('index',axis=1)

In [54]:
# Split all origin land use 2 = retail by previous activity type
residential_mu_list=['HOME','WORK_FROM_HOME']
retail_residential_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type'].isin(residential_mu_list)].reset_index().drop('index',axis=1)
restaurants_mu_list=['EAT', 'SOCIAL','OTHER_ACTIVITY_TYPE']
retail_restaurants_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type'].isin(restaurants_mu_list)].reset_index().drop('index',axis=1)
retail_shopping_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type']=='SHOP'].reset_index().drop('index',axis=1)
work_mu_list=['WORK','MAINTENANCE']
retail_work_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type'].isin(work_mu_list)].reset_index().drop('index',axis=1)
retail_park_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type']=='RECREATION'].reset_index().drop('index',axis=1)
retail_education_mu_df=replica_retail_mu_df[replica_retail_mu_df['previous_activity_type']=='SCHOOL'].reset_index().drop('index',axis=1)

## RESIDENTIAL 

In [55]:
# Filter all the RESIDENTIAL use buildings (based on origin land use 1)
replica_residential_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='residential'].reset_index().drop('index',axis=1)

In [56]:
# Drop RESIDENTIAL use trips to check all of them assigned at the end
drop_list=['residential']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [57]:
# Merge de residential use buildings with the mixed use ones with origin land use 2 = residential and  origin land use 2 = retail with previous activity HOME / WORK FROM HOME
replica_residential_df=pd.concat([replica_residential_df,replica_residential_mu_df,retail_residential_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_residential_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,origin_land_use_l2,destination_land_use_l1,destination_land_use_l2,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,1.761928e+18,42.392195,-71.123773,42.391419,-71.122663,EAT,HOME,02:31:22,02:33:59,2,...,multi_family,commercial,office,NaN,NaN,NaN,2,224-222@62j-sk2-tqf,42.389247,-71.120337
1,7.291174e+18,42.376795,-71.109011,42.372917,-71.119156,EAT,HOME,02:39:53,02:45:37,5,...,multi_family,commercial,retail,NaN,NaN,NaN,2,227-224@62j-sjv-b6k,42.372484,-71.117852
2,1.990876e+18,42.373705,-71.108024,42.372895,-71.117919,EAT,HOME,02:39:53,02:54:53,15,...,multi_family,commercial,retail,NaN,NaN,NaN,2,227-224@62j-sjv-b6k,42.372484,-71.117852
3,1.486862e+19,42.356004,-71.095598,42.359113,-71.092946,EAT,HOME,03:05:00,03:05:41,0,...,single_family,civic_institutional,education,NaN,NaN,NaN,3,222-222@62j-sj4-vcq,42.360213,-71.085994
4,7.940302e+18,42.356947,-71.095621,42.359185,-71.088175,EAT,HOME,03:05:00,03:07:59,2,...,single_family,civic_institutional,education,NaN,NaN,NaN,3,222-223@62j-sj2-dqf,42.361649,-71.090470


In [754]:
# Load the RESIDENTIAL shapefile data
filepath_residential_csv = filepath
residential_df = pd.read_csv(filepath_residential_csv + 'residentialbuildings.csv')
residential_df.head()

,LUDesc,Category,lon_POI,lat_POI
0,TWO-FAM-RES,Residential,-71.13426439,42.40229610
1,TWO-FAM-RES,Residential,-71.13377309,42.40226718
2,TWO-FAM-RES,Residential,-71.13362075,42.40222404
3,TWO-FAM-RES,Residential,-71.13348338,42.40215716
4,TWO-FAM-RES,Residential,-71.13435273,42.40218781


In [755]:
# Based on the trip origin lat,lon assign the closest RESIDENTIAL building with function KDTree
pd.set_option("display.precision", 8)
replica_residential_df['end_latitude']=''
replica_residential_df['end_longitude']=''
Z = np.array(residential_df[['lat_POI', 'lon_POI']])
tree = spatial.KDTree(Z)
for i in range(len(replica_residential_df)):
    dist, idxs = tree.query(np.array(replica_residential_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_residential_df.at[i, 'end_latitude'] = residential_df.at[idxs,'lat_POI']
    replica_residential_df.at[i, 'end_longitude'] = residential_df.at[idxs,'lon_POI']
replica_residential_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.66230097e+19,42.38829299,-71.12392811,42.371951,-71.085773,EAT,SOCIAL,00:21:55,00:28:02,6,...,TRAVEL,NaN,NaN,NaN,0,285-222@62j-sj2-x89,42.372127,-71.086885,42.388293,-71.123928
1,1.67153631e+19,42.37212200,-71.08712800,42.372097,-71.086866,EAT,EAT,00:45:13,00:45:32,0,...,TRAVEL,NaN,NaN,NaN,0,285-222@62j-sj2-x89,42.372127,-71.086885,42.372111,-71.087127
2,2.74441125e+18,42.36446953,-71.10084610,42.362556,-71.098477,EAT,SOCIAL,00:47:20,00:52:15,4,...,TRAVEL,NaN,NaN,NaN,0,zzw-223@62j-sj2-s3q,42.362611,-71.098646,42.36447,-71.100846
3,5.70534856e+18,42.37135984,-71.11485080,42.371330,-71.116470,EAT,HOME,00:59:15,01:01:23,2,...,TRAVEL,NaN,NaN,NaN,0,223-222@62j-sjv-9xq,42.370282,-71.118519,42.37136,-71.114851
4,2.67984415e+18,42.37135984,-71.11485080,42.371330,-71.116470,EAT,HOME,00:59:15,01:02:20,3,...,TRAVEL,NaN,NaN,NaN,0,223-222@62j-sjv-9xq,42.370282,-71.118519,42.37136,-71.114851


In [756]:
# Save the relevant data for the GAMA simulation model
residential_trips_df=replica_residential_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                            'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
residential_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.66230097e+19,42.372127,-71.086885,42.388293,-71.123928,EAT,00:21:55,00:28:02,6,2.92798766,0
1,1.67153631e+19,42.372127,-71.086885,42.372111,-71.087127,EAT,00:45:13,00:45:32,0,0.01652482,0
2,2.74441125e+18,42.362611,-71.098646,42.36447,-71.100846,EAT,00:47:20,00:52:15,4,0.25483028,0
3,5.70534856e+18,42.370282,-71.118519,42.37136,-71.114851,EAT,00:59:15,01:01:23,2,0.11117711,0
4,2.67984415e+18,42.370282,-71.118519,42.37136,-71.114851,EAT,00:59:15,01:02:20,3,0.15972146,0


## INDUSTRIAL

In [757]:
# Filter all the INDUSTRIAL use buildings (based on origin land use 1)
replica_industrial_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='industrial'].reset_index().drop('index',axis=1)

In [758]:
# Drop INDUSTRIAL use trips to check if all of them are assigned at the end
drop_list=['industrial']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False].reset_index().drop('index', axis=1)

In [759]:
# Merge the INDUSTRIAL use buildings with the mixed use ones with origin land use 2 = industrial
replica_industrial_df=pd.concat([replica_industrial_df,replica_industrial_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_industrial_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,6.67862780e+18,42.36497600,-71.08717200,42.361582,-71.086395,EAT,RECREATION,05:12:26,05:14:02,1,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470
1,1.23989003e+19,42.36439300,-71.08209700,42.364176,-71.083504,EAT,EAT,05:54:41,05:56:20,1,...,mixed_use,retail,TRAVEL,NaN,NaN,NaN,5,225-223@62j-sj3-3bk,42.364512,-71.082035
2,1.78664711e+19,42.36439300,-71.08209700,42.363323,-71.082471,EAT,EAT,05:54:41,05:58:22,3,...,mixed_use,non_retail_attraction,TRAVEL,NaN,NaN,NaN,5,225-223@62j-sj3-3bk,42.364512,-71.082035
3,3.95862208e+18,42.36157407,-71.09844808,42.361746,-71.097075,EAT,WORK,06:17:05,06:19:12,2,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,6,23d-222@62j-sj2-s5z,42.364025,-71.100857
4,1.44697365e+19,42.36472000,-71.08214500,42.372033,-71.074617,EAT,EAT,06:56:52,06:59:23,2,...,mixed_use,office,TRAVEL,NaN,NaN,NaN,6,zzw-223@62j-shx-gx5,42.371960,-71.075408


In [760]:
# SafeGraph Data for industrial buildings
industrial_poi_list=['Other Amusement and Recreation Industries','Other Miscellaneous Manufacturing','Gambling Industries',
                    'Coating, Engraving, Heat Treating, and Allied Activities','Machinery, Equipment, and Supplies Merchant Wholesalers',
                    'Motion Picture and Video Industries','Converted Paper Product Manufacturing','Glass and Glass Product Manufacturing',
                    'Electric Power Generation, Transmission and Distribution','Beverage Manufacturing', 'Sound Recording Industries',
                    'Bakeries and Tortilla Manufacturing']
industrial_sg_df=places_df[places_df['top_category'].isin(industrial_poi_list)]
drop_list=['open_hours','location_name','index']
industrial_sg_df=industrial_sg_df.reset_index().drop(drop_list,axis=1)
industrial_sg_df.head()

,placekey,latitude,longitude,top_category
0,22n-222@62j-sjy-b6k,42.376361,-71.145118,Other Amusement and Recreation Industries
1,zzz-222@62j-sjt-85f,42.396696,-71.137653,Other Amusement and Recreation Industries
2,228-22j@62j-sj2-k4v,42.374884,-71.104421,Other Amusement and Recreation Industries
3,222-222@62j-sjs-gkz,42.393618,-71.148304,Other Miscellaneous Manufacturing
4,22s-222@62j-sjs-vvf,42.394887,-71.127954,Other Amusement and Recreation Industries


In [761]:
# Based on the trip origin lat,lon assign the closest INDUSTRIAL building with function KDTree
pd.set_option("display.precision", 8)
replica_industrial_df['end_latitude']=''
replica_industrial_df['end_longitude']=''
Z = np.array(industrial_sg_df[['latitude', 'longitude']])
tree = spatial.KDTree(Z)
for i in range(len(replica_industrial_df)):
    dist, idxs = tree.query(np.array(replica_industrial_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_industrial_df.at[i, 'end_latitude'] = industrial_sg_df.at[idxs,'latitude']
    replica_industrial_df.at[i, 'end_longitude'] = industrial_sg_df.at[idxs,'longitude']
replica_industrial_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,6.67862780e+18,42.36497600,-71.08717200,42.361582,-71.086395,EAT,RECREATION,05:12:26,05:14:02,1,...,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470,42.365003,-71.087029
1,1.23989003e+19,42.36439300,-71.08209700,42.364176,-71.083504,EAT,EAT,05:54:41,05:56:20,1,...,TRAVEL,NaN,NaN,NaN,5,225-223@62j-sj3-3bk,42.364512,-71.082035,42.364369,-71.081379
2,1.78664711e+19,42.36439300,-71.08209700,42.363323,-71.082471,EAT,EAT,05:54:41,05:58:22,3,...,TRAVEL,NaN,NaN,NaN,5,225-223@62j-sj3-3bk,42.364512,-71.082035,42.364369,-71.081379
3,3.95862208e+18,42.36157407,-71.09844808,42.361746,-71.097075,EAT,WORK,06:17:05,06:19:12,2,...,TRAVEL,NaN,NaN,NaN,6,23d-222@62j-sj2-s5z,42.364025,-71.100857,42.361405,-71.098085
4,1.44697365e+19,42.36472000,-71.08214500,42.372033,-71.074617,EAT,EAT,06:56:52,06:59:23,2,...,TRAVEL,NaN,NaN,NaN,6,zzw-223@62j-shx-gx5,42.371960,-71.075408,42.364369,-71.081379


In [762]:
# Save the relevant data for the GAMA simulation model
industrial_trips_df=replica_industrial_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                           'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
industrial_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,6.67862780e+18,42.361649,-71.090470,42.365003,-71.087029,EAT,05:12:26,05:14:02,1,0.52709554,5
1,1.23989003e+19,42.364512,-71.082035,42.364369,-71.081379,EAT,05:54:41,05:56:20,1,0.08581394,5
2,1.78664711e+19,42.364512,-71.082035,42.364369,-71.081379,EAT,05:54:41,05:58:22,3,0.19096201,5
3,3.95862208e+18,42.364025,-71.100857,42.361405,-71.098085,EAT,06:17:05,06:19:12,2,0.11027069,6
4,1.44697365e+19,42.371960,-71.075408,42.364369,-71.081379,EAT,06:56:52,06:59:23,2,1.08563758,6


## OFFICE 

In [763]:
# Merge de OFFICE use buildings: mixed use and commercial ones with origin land use 2 = office
replica_office_df=pd.concat([replica_office_mu_df,replica_office_c_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_office_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,9.10506371e+18,42.38876400,-71.11988600,42.389225,-71.120331,EAT,EAT,04:32:31,04:33:26,0,...,commercial,retail,TRAVEL,NaN,NaN,NaN,4,224-222@62j-sk2-tqf,42.389247,-71.120337
1,9.24926302e+18,42.37221666,-71.13393439,42.371330,-71.116470,EAT,WORK,05:58:18,06:06:04,7,...,commercial,retail,TRAVEL,NaN,NaN,NaN,5,228-223@62j-sjv-9zz,42.372027,-71.120423
2,1.03393589e+19,42.36245700,-71.09401400,42.362658,-71.094274,EAT,MAINTENANCE,06:09:44,06:10:43,0,...,mixed_use,office,TRAVEL,NaN,NaN,NaN,6,227-222@62j-sj2-ffz,42.363082,-71.096974
3,1.45081541e+19,42.36905917,-71.10885112,42.368687,-71.109925,EAT,HOME,06:12:48,06:14:24,1,...,mixed_use,multi_family,TRAVEL,NaN,NaN,NaN,6,2b4-222@62j-sj2-jqf,42.368421,-71.109209
4,9.20814749e+17,42.36579400,-71.10483000,42.372033,-71.074617,EAT,SHOP,06:15:50,06:21:36,5,...,mixed_use,office,TRAVEL,NaN,NaN,NaN,6,zzw-223@62j-shx-gx5,42.371960,-71.075408


In [764]:
# SafeGraph Data for OFFICE buildings
office_poi_list=['Offices of Real Estate Agents and Brokers','Management of Companies and Enterprises','Agencies, Brokerages, and Other Insurance Related Activities',
                    'Electronic and Precision Equipment Repair and Maintenance','Architectural, Engineering, and Related Services',
                    'Personal and Household Goods Repair and Maintenance','Other Professional, Scientific, and Technical Services',
                    'Building Equipment Contractors','Automobile Dealers','Activities Related to Real Estate','Other Financial Investment Activities',
                    'Travel Arrangement and Reservation Services','Radio and Television Broadcasting','Automotive Equipment Rental and Leasing',
                    'Management, Scientific, and Technical Consulting Services','Building Material and Supplies Dealers',
                    'Consumer Goods Rental','Building Finishing Contractors','Couriers and Express Delivery Services',
                    'Cable and Other Subscription Programming','Advertising, Public Relations, and Related Services',
                    'Administration of Human Resource Programs','Other Specialty Trade Contractors']
office_sg_df=places_df[places_df['top_category'].isin(office_poi_list)]
drop_list=['open_hours','location_name','index']
office_sg_df=office_sg_df.reset_index().drop(drop_list,axis=1)
office_sg_df.head()

,placekey,latitude,longitude,top_category
0,zzw-222@62j-shx-hbk,42.371713,-71.074103,Offices of Real Estate Agents and Brokers
1,227-223@62j-sj3-2tv,42.363408,-71.081699,Management of Companies and Enterprises
2,25b-222@62j-sj2-qs5,42.359334,-71.106159,"Agencies, Brokerages, and Other Insurance Rela..."
3,226-225@62j-sjt-68v,42.388141,-71.132045,Electronic and Precision Equipment Repair and ...
4,22g-222@62j-sjv-ndv,42.373906,-71.124891,"Architectural, Engineering, and Related Services"


In [765]:
# Based on the trip origin lat,lon assign the closest OFFICE building with function KDTree
pd.set_option("display.precision", 8)
replica_office_df['end_latitude']=''
replica_office_df['end_longitude']=''
Z = np.array(office_sg_df[['latitude', 'longitude']])
tree = spatial.KDTree(Z)
for i in range(len(replica_office_df)):
    dist, idxs = tree.query(np.array(replica_office_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_office_df.at[i, 'end_latitude'] = office_sg_df.at[idxs,'latitude']
    replica_office_df.at[i, 'end_longitude'] = office_sg_df.at[idxs,'longitude']
replica_office_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,9.10506371e+18,42.38876400,-71.11988600,42.389225,-71.120331,EAT,EAT,04:32:31,04:33:26,0,...,TRAVEL,NaN,NaN,NaN,4,224-222@62j-sk2-tqf,42.389247,-71.120337,42.389423,-71.119749
1,9.24926302e+18,42.37221666,-71.13393439,42.371330,-71.116470,EAT,WORK,05:58:18,06:06:04,7,...,TRAVEL,NaN,NaN,NaN,5,228-223@62j-sjv-9zz,42.372027,-71.120423,42.373906,-71.124891
2,1.03393589e+19,42.36245700,-71.09401400,42.362658,-71.094274,EAT,MAINTENANCE,06:09:44,06:10:43,0,...,TRAVEL,NaN,NaN,NaN,6,227-222@62j-sj2-ffz,42.363082,-71.096974,42.361778,-71.094514
3,1.45081541e+19,42.36905917,-71.10885112,42.368687,-71.109925,EAT,HOME,06:12:48,06:14:24,1,...,TRAVEL,NaN,NaN,NaN,6,2b4-222@62j-sj2-jqf,42.368421,-71.109209,42.368958,-71.10884
4,9.20814749e+17,42.36579400,-71.10483000,42.372033,-71.074617,EAT,SHOP,06:15:50,06:21:36,5,...,TRAVEL,NaN,NaN,NaN,6,zzw-223@62j-shx-gx5,42.371960,-71.075408,42.365631,-71.104452


In [766]:
# Save the relevant data for the GAMA simulation model
office_trips_df=replica_office_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                   'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
office_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,9.10506371e+18,42.389247,-71.120337,42.389423,-71.119749,EAT,04:32:31,04:33:26,0,0.04756423,4
1,9.24926302e+18,42.372027,-71.120423,42.373906,-71.124891,EAT,05:58:18,06:06:04,7,1.33276108,5
2,1.03393589e+19,42.363082,-71.096974,42.361778,-71.094514,EAT,06:09:44,06:10:43,0,0.05117650,6
3,1.45081541e+19,42.368421,-71.109209,42.368958,-71.10884,EAT,06:12:48,06:14:24,1,0.08306649,6
4,9.20814749e+17,42.371960,-71.075408,42.365631,-71.104452,EAT,06:15:50,06:21:36,5,2.75007372,6


## SHOP 

In [767]:
# Merge de SHOP use buildings: mixed use and commercial ones with origin land use 2 = retail and with previous activity SHOP
replica_shop_df=pd.concat([retail_shopping_c_df,retail_shopping_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_shop_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,1.20039641e+19,42.368357,-71.075978,42.369373,-71.077051,EAT,SHOP,04:09:22,04:11:08,1,...,commercial,office,TRAVEL,NaN,NaN,NaN,4,zzw-222@62j-shx-kcq,42.369817,-71.077794
1,7.05405313e+18,42.365047,-71.102806,42.372810,-71.117560,EAT,SHOP,06:28:15,06:31:57,3,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,6,zzy-224@62j-sjv-85f,42.373371,-71.118965
2,1.53765441e+19,42.372461,-71.119544,42.372122,-71.087128,EAT,SHOP,06:33:07,06:47:58,14,...,mixed_use,multi_family,TRAVEL,NaN,NaN,NaN,6,22d-222@62j-sj2-vzz,42.371281,-71.083318
3,9.94202805e+18,42.365487,-71.103465,42.365675,-71.103734,EAT,SHOP,06:39:30,06:40:15,0,...,commercial,retail,TRAVEL,NaN,NaN,NaN,6,22j-222@62j-sj2-rkz,42.365675,-71.103734
4,6.72337688e+18,42.389977,-71.117999,42.389362,-71.119548,EAT,SHOP,06:55:15,06:59:18,4,...,commercial,retail,TRAVEL,NaN,NaN,NaN,6,235-223@62j-sk2-vmk,42.388388,-71.118232


In [768]:
# SafeGraph Data for SHOPPING buildings
shop_poi_list=['Other Miscellaneous Store Retailers','Jewelry, Luggage, and Leather Goods Stores','Clothing Stores',
              'Office Supplies, Stationery, and Gift Stores','Furniture Stores','Beer, Wine, and Liquor Stores','Grocery Stores',
              'Specialty Food Stores','Shoe Stores','Florists','Health and Personal Care Stores','Printing and Related Support Activities',
              'Home Furnishings Stores','Electronics and Appliance Stores','Department Stores','Used Merchandise Stores',
              "Drugs and Druggists' Sundries Merchant Wholesalers",'General Merchandise Stores, including Warehouse Clubs and Supercenters',
              'Lawn and Garden Equipment and Supplies Stores','Other Motor Vehicle Dealers','Book Stores and News Dealers']
shop_sg_df=places_df[places_df['top_category'].isin(shop_poi_list)]
drop_list=['open_hours','location_name','index']
shop_sg_df=shop_sg_df.reset_index().drop(drop_list,axis=1)
shop_sg_df.head()

,placekey,latitude,longitude,top_category
0,225-226@62j-sk2-yn5,42.391647,-71.123117,Other Miscellaneous Store Retailers
1,zzw-222@62j-sj2-rkz,42.365073,-71.102597,Grocery Stores
2,22k-222@62j-sj2-c3q,42.362380,-71.098321,Book Stores and News Dealers
3,22r-223@62j-sjs-vpv,42.394297,-71.126183,Grocery Stores
4,zzw-226@62j-sjv-9mk,42.373575,-71.120467,"Jewelry, Luggage, and Leather Goods Stores"


In [769]:
# Based on the trip origin lat,lon assign the closest SHOPPING building with function KDTree
pd.set_option("display.precision", 8)
replica_shop_df['end_latitude']=''
replica_shop_df['end_longitude']=''
Z = np.array(shop_sg_df[['latitude', 'longitude']])
tree = spatial.KDTree(Z)
for i in range(len(replica_shop_df)):
    dist, idxs = tree.query(np.array(replica_shop_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_shop_df.at[i, 'end_latitude'] = shop_sg_df.at[idxs,'latitude']
    replica_shop_df.at[i, 'end_longitude'] = shop_sg_df.at[idxs,'longitude']
replica_shop_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.20039641e+19,42.368357,-71.075978,42.369373,-71.077051,EAT,SHOP,04:09:22,04:11:08,1,...,TRAVEL,NaN,NaN,NaN,4,zzw-222@62j-shx-kcq,42.369817,-71.077794,42.368351,-71.075953
1,7.05405313e+18,42.365047,-71.102806,42.372810,-71.117560,EAT,SHOP,06:28:15,06:31:57,3,...,TRAVEL,NaN,NaN,NaN,6,zzy-224@62j-sjv-85f,42.373371,-71.118965,42.365178,-71.102759
2,1.53765441e+19,42.372461,-71.119544,42.372122,-71.087128,EAT,SHOP,06:33:07,06:47:58,14,...,TRAVEL,NaN,NaN,NaN,6,22d-222@62j-sj2-vzz,42.371281,-71.083318,42.372399,-71.11943
3,9.94202805e+18,42.365487,-71.103465,42.365675,-71.103734,EAT,SHOP,06:39:30,06:40:15,0,...,TRAVEL,NaN,NaN,NaN,6,22j-222@62j-sj2-rkz,42.365675,-71.103734,42.365566,-71.103333
4,6.72337688e+18,42.389977,-71.117999,42.389362,-71.119548,EAT,SHOP,06:55:15,06:59:18,4,...,TRAVEL,NaN,NaN,NaN,6,235-223@62j-sk2-vmk,42.388388,-71.118232,42.389998,-71.118292


In [770]:
# Save the relevant data for the GAMA simulation model
shopping_trips_df=replica_shop_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                   'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
shopping_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.20039641e+19,42.369817,-71.077794,42.368351,-71.075953,EAT,04:09:22,04:11:08,1,0.09183881,4
1,7.05405313e+18,42.373371,-71.118965,42.365178,-71.102759,EAT,06:28:15,06:31:57,3,1.39050595,6
2,1.53765441e+19,42.371281,-71.083318,42.372399,-71.11943,EAT,06:33:07,06:47:58,14,1.88275453,6
3,9.94202805e+18,42.365675,-71.103734,42.365566,-71.103333,EAT,06:39:30,06:40:15,0,0.03898391,6
4,6.72337688e+18,42.388388,-71.118232,42.389998,-71.118292,EAT,06:55:15,06:59:18,4,0.21036179,6


### HOTELS

In [771]:
# Merge HOTEL use trips: commercial ones with origin land use 2 = retail and with previous activity LODGING
replica_hotel_df=retail_hotel_c_df
replica_hotel_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,1.38419628e+19,42.375718,-71.110876,42.364155,-71.102966,EAT,LODGING,00:13:11,00:36:32,23,...,mixed_use,retail,TRAVEL,NaN,NaN,NaN,0,24d-222@62j-sj2-s5z,42.363664,-71.101451
1,9.71430163e+17,42.373218,-71.081673,42.371941,-71.085644,EAT,LODGING,07:09:39,07:11:01,1,...,mixed_use,multi_family,TRAVEL,NaN,NaN,NaN,7,zzy-222@62j-sj2-vmk,42.371750,-71.084037
2,1.58710300e+19,42.373218,-71.081673,42.371951,-71.085773,EAT,LODGING,07:52:47,07:54:25,1,...,commercial,retail,TRAVEL,NaN,NaN,NaN,7,zzy-222@62j-sj2-vmk,42.371750,-71.084037
3,1.57291604e+19,42.372998,-71.079934,42.371941,-71.085644,EAT,LODGING,07:52:47,08:03:55,11,...,mixed_use,multi_family,TRAVEL,NaN,NaN,NaN,7,zzy-222@62j-sj2-vmk,42.371750,-71.084037
4,4.66562756e+18,42.362076,-71.100070,42.369563,-71.111341,EAT,LODGING,08:00:11,08:02:30,2,...,commercial,retail,TRAVEL,NaN,NaN,NaN,8,25w-226@62j-sjv-j35,42.369628,-71.111400


In [772]:
# SafeGraph Data for HOTEL buildings
hotel_poi_list=['Traveler Accommodation']
hotel_sg_df=places_df[places_df['top_category'].isin(hotel_poi_list)]
drop_list=['open_hours','location_name','index']
hotel_sg_df=hotel_sg_df.reset_index().drop(drop_list,axis=1)
hotel_sg_df.head()

,placekey,latitude,longitude,top_category
0,22m-222@62j-shx-tjv,42.367994,-71.075304,Traveler Accommodation
1,222-222@62j-sjr-zcq,42.388733,-71.142343,Traveler Accommodation
2,223-222@62j-sj2-dvz,42.362149,-71.087252,Traveler Accommodation
3,24z-222@62j-sk2-qfz,42.375492,-71.110654,Traveler Accommodation
4,225-225@62j-shx-t35,42.366778,-71.074993,Traveler Accommodation


In [773]:
# Based on the trip origin lat,lon assign the closest HOTEL building with function KDTree
pd.set_option("display.precision", 8)
replica_hotel_df['end_latitude']=''
replica_hotel_df['end_longitude']=''
Z = np.array(hotel_sg_df[['latitude', 'longitude']])
tree = spatial.KDTree(Z)
for i in range(len(replica_hotel_df)):
    dist, idxs = tree.query(np.array(replica_hotel_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_hotel_df.at[i, 'end_latitude'] = hotel_sg_df.at[idxs,'latitude']
    replica_hotel_df.at[i, 'end_longitude'] = hotel_sg_df.at[idxs,'longitude']
replica_hotel_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.38419628e+19,42.375718,-71.110876,42.364155,-71.102966,EAT,LODGING,00:13:11,00:36:32,23,...,TRAVEL,NaN,NaN,NaN,0,24d-222@62j-sj2-s5z,42.363664,-71.101451,42.375492,-71.110654
1,9.71430163e+17,42.373218,-71.081673,42.371941,-71.085644,EAT,LODGING,07:09:39,07:11:01,1,...,TRAVEL,NaN,NaN,NaN,7,zzy-222@62j-sj2-vmk,42.371750,-71.084037,42.373197,-71.081699
2,1.58710300e+19,42.373218,-71.081673,42.371951,-71.085773,EAT,LODGING,07:52:47,07:54:25,1,...,TRAVEL,NaN,NaN,NaN,7,zzy-222@62j-sj2-vmk,42.371750,-71.084037,42.373197,-71.081699
3,1.57291604e+19,42.372998,-71.079934,42.371941,-71.085644,EAT,LODGING,07:52:47,08:03:55,11,...,TRAVEL,NaN,NaN,NaN,7,zzy-222@62j-sj2-vmk,42.371750,-71.084037,42.372955,-71.079939
4,4.66562756e+18,42.362076,-71.100070,42.369563,-71.111341,EAT,LODGING,08:00:11,08:02:30,2,...,TRAVEL,NaN,NaN,NaN,8,25w-226@62j-sjv-j35,42.369628,-71.111400,42.362183,-71.100137


In [774]:
# Save the relevant data for the GAMA simulation model
hotel_trips_df=replica_hotel_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                 'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
hotel_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.38419628e+19,42.363664,-71.101451,42.375492,-71.110654,EAT,00:13:11,00:36:32,23,1.20961137,0
1,9.71430163e+17,42.371750,-71.084037,42.373197,-71.081699,EAT,07:09:39,07:11:01,1,0.53826544,7
2,1.58710300e+19,42.371750,-71.084037,42.373197,-71.081699,EAT,07:52:47,07:54:25,1,0.54040161,7
3,1.57291604e+19,42.371750,-71.084037,42.372955,-71.079939,EAT,07:52:47,08:03:55,11,0.57654065,7
4,4.66562756e+18,42.369628,-71.111400,42.362183,-71.100137,EAT,08:00:11,08:02:30,2,0.85550441,8


### RESTAURANTS

In [775]:
# Merge the RESTAURANT buildings: mixed use and commercial ones with origin land use 2 = retail with previous activity EAT, SOCIAL, OTHER_ACTIVITY_TYPE
replica_restaurants_df=pd.concat([retail_restaurants_mu_df,retail_restaurants_c_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_restaurants_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,1.62290058e+19,42.3720970,-71.08686600,42.372122,-71.087128,EAT,EAT,00:34:13,00:34:32,0,...,mixed_use,multi_family,TRAVEL,NaN,NaN,NaN,0,285-222@62j-sj2-x89,42.372127,-71.086885
1,9.24549446e+18,42.3646724,-71.08384421,42.361582,-71.086395,EAT,SOCIAL,05:52:49,06:01:13,8,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470
2,5.38198951e+18,42.3732350,-71.11911800,42.373076,-71.118722,EAT,EAT,06:05:35,06:06:12,0,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,6,zzy-224@62j-sjv-85f,42.373371,-71.118965
3,1.50531769e+19,42.3825180,-71.11995800,42.374492,-71.120011,EAT,EAT,06:28:11,06:29:17,1,...,commercial,retail,TRAVEL,NaN,NaN,NaN,6,zzw-222@62j-sjv-975,42.374189,-71.121151
4,6.99800800e+18,42.3719680,-71.12029400,42.371436,-71.116713,EAT,EAT,06:32:40,06:34:28,1,...,commercial,retail,TRAVEL,NaN,NaN,NaN,6,228-223@62j-sjv-9zz,42.372027,-71.120423


In [776]:
# SafeGraph Data for RESTAURANT buildings
restaurant_poi_list=['Restaurants and Other Eating Places','Drinking Places (Alcoholic Beverages)','Special Food Services']
restaurant_sg_df=places_df[places_df['top_category'].isin(restaurant_poi_list)]
drop_list=['open_hours','location_name','index']
restaurant_sg_df=restaurant_sg_df.reset_index().drop(drop_list,axis=1)
restaurant_sg_df.head()

,placekey,latitude,longitude,top_category
0,zzw-222@62j-sj2-s3q,42.362221,-71.098786,Restaurants and Other Eating Places
1,zzw-224@62j-sjv-975,42.374318,-71.122075,Restaurants and Other Eating Places
2,224-222@62j-sjv-9zz,42.372089,-71.121409,Restaurants and Other Eating Places
3,233-223@62j-sj3-9mk,42.374045,-71.100616,Restaurants and Other Eating Places
4,zzw-224@62j-sgg-kvf,42.360979,-71.113912,Restaurants and Other Eating Places


In [777]:
# Based on the trip origin lat,lon assign the closest RESTAURANT building with function KDTree
pd.set_option("display.precision", 8)
replica_restaurants_df['end_latitude']=''
replica_restaurants_df['end_longitude']=''
Z = np.array(restaurant_sg_df[['latitude', 'longitude']])
tree = spatial.KDTree(Z)
for i in range(len(replica_restaurants_df)):
    dist, idxs = tree.query(np.array(replica_restaurants_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_restaurants_df.at[i, 'end_latitude'] = restaurant_sg_df.at[idxs,'latitude']
    replica_restaurants_df.at[i, 'end_longitude'] = restaurant_sg_df.at[idxs,'longitude']
replica_restaurants_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.62290058e+19,42.3720970,-71.08686600,42.372122,-71.087128,EAT,EAT,00:34:13,00:34:32,0,...,TRAVEL,NaN,NaN,NaN,0,285-222@62j-sj2-x89,42.372127,-71.086885,42.372127,-71.086885
1,9.24549446e+18,42.3646724,-71.08384421,42.361582,-71.086395,EAT,SOCIAL,05:52:49,06:01:13,8,...,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470,42.364305,-71.084033
2,5.38198951e+18,42.3732350,-71.11911800,42.373076,-71.118722,EAT,EAT,06:05:35,06:06:12,0,...,TRAVEL,NaN,NaN,NaN,6,zzy-224@62j-sjv-85f,42.373371,-71.118965,42.373276,-71.119212
3,1.50531769e+19,42.3825180,-71.11995800,42.374492,-71.120011,EAT,EAT,06:28:11,06:29:17,1,...,TRAVEL,NaN,NaN,NaN,6,zzw-222@62j-sjv-975,42.374189,-71.121151,42.382519,-71.119948
4,6.99800800e+18,42.3719680,-71.12029400,42.371436,-71.116713,EAT,EAT,06:32:40,06:34:28,1,...,TRAVEL,NaN,NaN,NaN,6,228-223@62j-sjv-9zz,42.372027,-71.120423,42.371963,-71.120281


In [778]:
# Save the relevant data for the GAMA simulation model
restaurant_trips_df=replica_restaurants_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                            'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
restaurant_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.62290058e+19,42.372127,-71.086885,42.372127,-71.086885,EAT,00:34:13,00:34:32,0,0.01652482,0
1,9.24549446e+18,42.361649,-71.090470,42.364305,-71.084033,EAT,05:52:49,06:01:13,8,0.43536091,5
2,5.38198951e+18,42.373371,-71.118965,42.373276,-71.119212,EAT,06:05:35,06:06:12,0,0.03244217,6
3,1.50531769e+19,42.374189,-71.121151,42.382519,-71.119948,EAT,06:28:11,06:29:17,1,0.62727397,6
4,6.99800800e+18,42.372027,-71.120423,42.371963,-71.120281,EAT,06:32:40,06:34:28,1,0.66559504,6


### WORK 

In [779]:
# Merge the WORK buildings: mixed use and commercial ones with origin land use 2 = retail with previous activity WORK
replica_work_df=pd.concat([retail_work_mu_df,retail_work_c_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_work_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,1.28635372e+19,42.37234632,-71.11605083,42.372303,-71.116061,EAT,WORK,05:48:03,05:48:05,0,...,commercial,retail,TRAVEL,NaN,NaN,NaN,5,zzw-222@62j-sjv-7yv,42.372708,-71.116945
1,1.45201968e+19,42.36205087,-71.08474694,42.361582,-71.086395,EAT,WORK,05:54:30,05:58:54,4,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470
2,1.21960461e+19,42.37267600,-71.08782000,42.372097,-71.086866,EAT,MAINTENANCE,06:18:49,06:23:14,4,...,mixed_use,retail,TRAVEL,NaN,NaN,NaN,6,22d-222@62j-sj2-vzz,42.371281,-71.083318
3,1.70664302e+19,42.37524000,-71.14771400,42.375451,-71.149096,EAT,MAINTENANCE,06:22:02,06:23:48,1,...,commercial,office,TRAVEL,NaN,NaN,NaN,6,22t-226@62j-sjy-7yv,42.375451,-71.149096
4,2.38533435e+18,42.37420197,-71.12002034,42.374228,-71.120007,EAT,WORK,06:22:13,06:22:14,0,...,mixed_use,retail,TRAVEL,NaN,NaN,NaN,6,zzw-222@62j-sjv-975,42.374189,-71.121151


In [780]:
# Concat the SHOPPING, HOTELS and RESTAURANTS shapefiles: because the people can work in any of these buildings
work_sg_df=pd.concat([shop_sg_df,hotel_sg_df,restaurant_sg_df]).reset_index().drop('index',axis=1)

# Based on the trip origin lat,lon assign the closest work building with function KDTree
pd.set_option("display.precision", 8)
replica_work_df['end_latitude']=''
replica_work_df['end_longitude']=''
Z = np.array(work_sg_df[['latitude', 'longitude']])
tree = spatial.KDTree(Z)
for i in range(len(replica_work_df)):
    dist, idxs = tree.query(np.array(replica_work_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_work_df.at[i, 'end_latitude'] = work_sg_df.at[idxs,'latitude']
    replica_work_df.at[i, 'end_longitude'] = work_sg_df.at[idxs,'longitude']
replica_work_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.28635372e+19,42.37234632,-71.11605083,42.372303,-71.116061,EAT,WORK,05:48:03,05:48:05,0,...,TRAVEL,NaN,NaN,NaN,5,zzw-222@62j-sjv-7yv,42.372708,-71.116945,42.372303,-71.116061
1,1.45201968e+19,42.36205087,-71.08474694,42.361582,-71.086395,EAT,WORK,05:54:30,05:58:54,4,...,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470,42.362077,-71.084833
2,1.21960461e+19,42.37267600,-71.08782000,42.372097,-71.086866,EAT,MAINTENANCE,06:18:49,06:23:14,4,...,TRAVEL,NaN,NaN,NaN,6,22d-222@62j-sj2-vzz,42.371281,-71.083318,42.372267,-71.088105
3,1.70664302e+19,42.37524000,-71.14771400,42.375451,-71.149096,EAT,MAINTENANCE,06:22:02,06:23:48,1,...,TRAVEL,NaN,NaN,NaN,6,22t-226@62j-sjy-7yv,42.375451,-71.149096,42.375309,-71.148852
4,2.38533435e+18,42.37420197,-71.12002034,42.374228,-71.120007,EAT,WORK,06:22:13,06:22:14,0,...,TRAVEL,NaN,NaN,NaN,6,zzw-222@62j-sjv-975,42.374189,-71.121151,42.374223,-71.120017


In [781]:
# Save the relevant data for the GAMA simulation model
work_trips_df=replica_work_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                               'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
work_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.28635372e+19,42.372708,-71.116945,42.372303,-71.116061,EAT,05:48:03,05:48:05,0,0.00187972,5
1,1.45201968e+19,42.361649,-71.090470,42.362077,-71.084833,EAT,05:54:30,05:58:54,4,0.22797026,5
2,1.21960461e+19,42.371281,-71.083318,42.372267,-71.088105,EAT,06:18:49,06:23:14,4,0.22888771,6
3,1.70664302e+19,42.375451,-71.149096,42.375309,-71.148852,EAT,06:22:02,06:23:48,1,0.09182993,6
4,2.38533435e+18,42.374189,-71.121151,42.374223,-71.120017,EAT,06:22:13,06:22:14,0,0.00168197,6


## NON RETAIL ATTRACTION 

In [782]:
# Merge the NON RETAIL ATTRACTION use buildings: mixed use ones with origin land use 2 = non retail
replica_non_retail_df=pd.concat([replica_non_retail_mu_df,replica_non_retail_c_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_non_retail_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,3.71345591e+18,42.36348128,-71.08253301,42.363651,-71.083131,EAT,HOME,00:05:50,00:06:27,0,...,mixed_use,non_retail_attraction,TRAVEL,NaN,NaN,NaN,0,23m-222@62j-shx-k4v,42.368530,-71.079323
1,2.98487207e+18,42.36651161,-71.07912695,42.366431,-71.077736,EAT,WORK,04:47:29,04:50:43,3,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,4,zzw-22j@62j-shx-tjv,42.368019,-71.075326
2,7.47735221e+18,42.36348128,-71.08253301,42.363377,-71.081452,EAT,HOME,05:34:00,05:35:36,1,...,mixed_use,non_retail_attraction,TRAVEL,NaN,NaN,NaN,5,22b-223@62j-sj3-2zf,42.363587,-71.083219
3,4.76310803e+18,42.36625388,-71.08331323,42.361582,-71.086395,EAT,HOME,05:38:49,05:50:32,11,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470
4,1.09018521e+19,42.36625388,-71.08331323,42.360092,-71.085936,EAT,HOME,05:38:49,05:41:06,2,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470


In [783]:
# SafeGraph Data for NON RETAIL ATTRACTION buildings
non_retail_poi_list=['Sporting Goods, Hobby, and Musical Instrument Stores','Museums, Historical Sites, and Similar Institutions',
                    'Amusement Parks and Arcades','Performing Arts Companies','Promoters of Performing Arts, Sports, and Similar Events',
                    'Religious Organizations']
non_retail_sg_df=places_df[places_df['top_category'].isin(non_retail_poi_list)]
drop_list=['open_hours','location_name','index']
non_retail_sg_df=non_retail_sg_df.reset_index().drop(drop_list,axis=1)
non_retail_sg_df.head()

,placekey,latitude,longitude,top_category
0,245-222@62j-sj2-5zz,42.366758,-71.097510,Religious Organizations
1,22g-223@62j-sj2-nqz,42.363877,-71.104361,Religious Organizations
2,zzy-224@62j-shx-jy9,42.368135,-71.076448,"Sporting Goods, Hobby, and Musical Instrument ..."
3,zzy-222@62j-sjv-fs5,42.364394,-71.111361,"Museums, Historical Sites, and Similar Institu..."
4,zzw-223@62j-sjv-j35,42.369868,-71.112096,"Museums, Historical Sites, and Similar Institu..."


In [784]:
# Based on the trip origin lat,lon assign the closest NON RETAIL ATTRACTION building with function KDTree
pd.set_option("display.precision", 8)
replica_non_retail_df['end_latitude']=''
replica_non_retail_df['end_longitude']=''
Z = np.array(non_retail_sg_df[['latitude', 'longitude']])
tree = spatial.KDTree(Z)
for i in range(len(replica_non_retail_df)):
    dist, idxs = tree.query(np.array(replica_non_retail_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_non_retail_df.at[i, 'end_latitude'] = non_retail_sg_df.at[idxs,'latitude']
    replica_non_retail_df.at[i, 'end_longitude'] = non_retail_sg_df.at[idxs,'longitude']
replica_non_retail_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,3.71345591e+18,42.36348128,-71.08253301,42.363651,-71.083131,EAT,HOME,00:05:50,00:06:27,0,...,TRAVEL,NaN,NaN,NaN,0,23m-222@62j-shx-k4v,42.368530,-71.079323,42.362934,-71.082109
1,2.98487207e+18,42.36651161,-71.07912695,42.366431,-71.077736,EAT,WORK,04:47:29,04:50:43,3,...,TRAVEL,NaN,NaN,NaN,4,zzw-22j@62j-shx-tjv,42.368019,-71.075326,42.365682,-71.079402
2,7.47735221e+18,42.36348128,-71.08253301,42.363377,-71.081452,EAT,HOME,05:34:00,05:35:36,1,...,TRAVEL,NaN,NaN,NaN,5,22b-223@62j-sj3-2zf,42.363587,-71.083219,42.362934,-71.082109
3,4.76310803e+18,42.36625388,-71.08331323,42.361582,-71.086395,EAT,HOME,05:38:49,05:50:32,11,...,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470,42.365952,-71.082562
4,1.09018521e+19,42.36625388,-71.08331323,42.360092,-71.085936,EAT,HOME,05:38:49,05:41:06,2,...,TRAVEL,NaN,NaN,NaN,5,222-223@62j-sj2-dqf,42.361649,-71.090470,42.365952,-71.082562


In [785]:
# Save the relevant data for the GAMA simulation model
non_retail_trips_df=replica_non_retail_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                         'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
non_retail_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,3.71345591e+18,42.368530,-71.079323,42.362934,-71.082109,EAT,00:05:50,00:06:27,0,0.03273589,0
1,2.98487207e+18,42.368019,-71.075326,42.365682,-71.079402,EAT,04:47:29,04:50:43,3,0.16815929,4
2,7.47735221e+18,42.363587,-71.083219,42.362934,-71.082109,EAT,05:34:00,05:35:36,1,0.08351108,5
3,4.76310803e+18,42.361649,-71.090470,42.365952,-71.082562,EAT,05:38:49,05:50:32,11,0.60707299,5
4,1.09018521e+19,42.361649,-71.090470,42.365952,-71.082562,EAT,05:38:49,05:41:06,2,1.09069964,5


## PARK 

In [786]:
# Filter all the OPEN SPACE use buildings (based on origin land use 1)
replica_park_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='open_space'].reset_index().drop('index',axis=1)

In [787]:
# Drop OPEN SPACE use trips to check if all of them are assigned at the end
drop_list=['open_space']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False]

In [788]:
# Merge de OPEN SPACE use buildings with the mixed use and commercial ones with origin land use 2 = open_space and  origin land use 2 = retail with previous activity RECREATION
replica_park_df=pd.concat([replica_park_df,retail_park_c_df,replica_park_mu_df,retail_park_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_park_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,5.09076771e+18,42.363428,-71.100044,42.362458,-71.098004,EAT,RECREATION,05:50:11,05:53:34,3,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,5,zzw-222@62j-sj2-bx5,42.361329,-71.096228
1,6.81136173e+18,42.388888,-71.119056,42.374306,-71.119126,EAT,RECREATION,06:51:51,06:54:44,2,...,commercial,office,TRAVEL,NaN,NaN,NaN,6,zzw-222@62j-sjv-975,42.374189,-71.121151
2,8.86366673e+18,42.364991,-71.102566,42.365028,-71.102423,EAT,RECREATION,07:16:29,07:17:10,0,...,commercial,retail,TRAVEL,NaN,NaN,NaN,7,22b-222@62j-sj2-rff,42.365028,-71.102423
3,1.18715710e+19,42.399843,-71.132420,42.389784,-71.118191,EAT,RECREATION,07:46:37,07:50:20,3,...,commercial,retail,TRAVEL,NaN,NaN,NaN,7,223-224@62j-sk2-y5f,42.390043,-71.118227
4,7.92169605e+18,42.373614,-71.109717,42.372053,-71.118300,EAT,SCHOOL,08:22:40,08:26:16,3,...,commercial,retail,TRAVEL,NaN,NaN,NaN,8,zzy-224@62j-sjv-85f,42.373371,-71.118965


In [789]:
# Load the PARK shapefile data
filepath_park_csv = "./Input Database/"
park_df = pd.read_csv(filepath_park_csv + 'parkbuildings.csv')
park_df.head()

,LUDesc,Category,lon_POI,lat_POI
0,Parklands,Public Open Space,-71.13838993,42.39974850
1,Parklands,Public Open Space,-71.13238731,42.39987779
2,Parklands,Public Open Space,-71.13271964,42.39967001
3,Parklands,Public Open Space,-71.13260403,42.39961282
4,Parklands,Public Open Space,-71.13247073,42.39782513


In [790]:
# Based on the trip origin lat,lon assign the closest PARK building with function KDTree
pd.set_option("display.precision", 8)
replica_park_df['end_latitude']=''
replica_park_df['end_longitude']=''
Z = np.array(park_df[['lat_POI', 'lon_POI']])
tree = spatial.KDTree(Z)
for i in range(len(replica_park_df)):
    dist, idxs = tree.query(np.array(replica_park_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_park_df.at[i, 'end_latitude'] = park_df.at[idxs,'lat_POI']
    replica_park_df.at[i, 'end_longitude'] = park_df.at[idxs,'lon_POI']
replica_park_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,5.09076771e+18,42.363428,-71.100044,42.362458,-71.098004,EAT,RECREATION,05:50:11,05:53:34,3,...,TRAVEL,NaN,NaN,NaN,5,zzw-222@62j-sj2-bx5,42.361329,-71.096228,42.362849,-71.099956
1,6.81136173e+18,42.388888,-71.119056,42.374306,-71.119126,EAT,RECREATION,06:51:51,06:54:44,2,...,TRAVEL,NaN,NaN,NaN,6,zzw-222@62j-sjv-975,42.374189,-71.121151,42.39103,-71.12178
2,8.86366673e+18,42.364991,-71.102566,42.365028,-71.102423,EAT,RECREATION,07:16:29,07:17:10,0,...,TRAVEL,NaN,NaN,NaN,7,22b-222@62j-sj2-rff,42.365028,-71.102423,42.362128,-71.10261
3,1.18715710e+19,42.399843,-71.132420,42.389784,-71.118191,EAT,RECREATION,07:46:37,07:50:20,3,...,TRAVEL,NaN,NaN,NaN,7,223-224@62j-sk2-y5f,42.390043,-71.118227,42.399878,-71.132387
4,7.92169605e+18,42.373614,-71.109717,42.372053,-71.118300,EAT,SCHOOL,08:22:40,08:26:16,3,...,TRAVEL,NaN,NaN,NaN,8,zzy-224@62j-sjv-85f,42.373371,-71.118965,42.369524,-71.10782


In [791]:
# Save the relevant data for the GAMA simulation model
park_trips_df=replica_park_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                               'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
park_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,5.09076771e+18,42.361329,-71.096228,42.362849,-71.099956,EAT,05:50:11,05:53:34,3,0.17562001,5
1,6.81136173e+18,42.374189,-71.121151,42.39103,-71.12178,EAT,06:51:51,06:54:44,2,1.05364622,6
2,8.86366673e+18,42.365028,-71.102423,42.362128,-71.10261,EAT,07:16:29,07:17:10,0,0.03554929,7
3,1.18715710e+19,42.390043,-71.118227,42.399878,-71.132387,EAT,07:46:37,07:50:20,3,1.39319049,7
4,7.92169605e+18,42.373371,-71.118965,42.369524,-71.10782,EAT,08:22:40,08:26:16,3,1.11282829,8


## TRANSPORTATION UTILITIES 

In [792]:
# Filter all the TRANSPORTATION UTILITIES use buildings (based on origin land use 1)
replica_transportationu_df=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1']=='transportation_utilities'].reset_index().drop('index',axis=1)

In [793]:
# Drop TRANSPORTATION UTILITIES use trips to check if all of them are assigned at the end
drop_list=['transportation_utilities']
eattrips_df_copy=eattrips_df_copy[eattrips_df_copy['origin_land_use_l1'].isin(drop_list)==False]

In [794]:
# Merge de TRANSPORTATION UTILITIES use buildings with the mixed use and commercial ones with origin land use 2 = open_space and  origin land use 2 = retail with previous activity RECREATION
replica_transportationu_df=pd.concat([replica_transportationu_df,replica_tu_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_transportationu_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,8.53350404e+18,42.36635300,-71.10604700,42.369340,-71.110758,EAT,WORK,08:52:57,09:00:52,7,...,commercial,retail,TRAVEL,NaN,NaN,NaN,8,25w-226@62j-sjv-j35,42.369628,-71.111400
1,1.03318534e+19,42.36311228,-71.07993720,42.364393,-71.082097,EAT,WORK,09:37:24,09:40:31,3,...,mixed_use,industrial,TRAVEL,NaN,NaN,NaN,9,23m-222@62j-shx-k4v,42.368530,-71.079323
2,1.32113948e+19,42.39713436,-71.12882474,42.396422,-71.129575,EAT,WORK,10:57:03,10:58:14,1,...,commercial,retail,TRAVEL,NaN,NaN,NaN,10,24m-222@62j-sjs-v75,42.397326,-71.130785
3,1.36145279e+19,42.37434300,-71.12105800,42.365028,-71.102423,EAT,SHOP,11:01:04,11:08:23,7,...,commercial,retail,TRAVEL,NaN,NaN,NaN,11,zzw-222@62j-sjp-jgk,42.365138,-71.102738
4,1.81992046e+19,42.36632415,-71.10608894,42.366071,-71.105327,EAT,WORK,11:36:25,11:38:09,1,...,commercial,retail,TRAVEL,NaN,NaN,NaN,11,22y-222@62j-sj2-t35,42.365915,-71.105008


In [795]:
# Load the transportation OSM data
list_osm_transportation=['bicycle_parking','bicycle_repair_station','bicycle_rental','boat_rental','boat_sharing','bus_station',
                         'car_rental','car_sharing','car_wash','vehicle_inspection','charging_station','ferry_terminal','fuel',
                         'grit_bin','motorcycle_parking','parking','parking_entrance','parking_space','taxi']
osm_transportation_df=osm_amenities_df[osm_amenities_df['amenity'].isin(list_osm_transportation)].reset_index().drop('index',axis=1)
osm_transportation_df.head()

,id,amenity,name,lat,lon
0,61429995,parking_entrance,NaN,42.3546475,-71.0684315
1,69504598,bus_station,Harvard Square,42.3733695,-71.1189903
2,313378293,parking,NaN,42.3676892,-71.0772986
3,325495023,parking_entrance,NaN,42.3743117,-71.1323594
4,328101383,parking_entrance,NaN,42.3958889,-71.1437272


In [796]:
# Based on the trip origin lat,lon assign the closest TRANSPORTATION UTILITY with function KDTree
pd.set_option("display.precision", 10)
replica_transportationu_df['end_latitude']=''
replica_transportationu_df['end_longitude']=''
Z = np.array(osm_transportation_df[['lat', 'lon']])
tree = spatial.KDTree(Z)
for i in range(len(replica_transportationu_df)):
    dist, idxs = tree.query(np.array(replica_transportationu_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_transportationu_df.at[i, 'end_latitude'] = osm_transportation_df.at[idxs,'lat']
    replica_transportationu_df.at[i, 'end_longitude'] = osm_transportation_df.at[idxs,'lon']
replica_transportationu_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,8.5335040444e+18,42.3663530000,-71.1060470000,42.369340,-71.110758,EAT,WORK,08:52:57,09:00:52,7,...,TRAVEL,NaN,NaN,NaN,8,25w-226@62j-sjv-j35,42.369628,-71.111400,42.3665288,-71.1057339
1,1.0331853431e+19,42.3631122825,-71.0799372005,42.364393,-71.082097,EAT,WORK,09:37:24,09:40:31,3,...,TRAVEL,NaN,NaN,NaN,9,23m-222@62j-shx-k4v,42.368530,-71.079323,42.3624471,-71.0800015
2,1.3211394794e+19,42.3971343586,-71.1288247412,42.396422,-71.129575,EAT,WORK,10:57:03,10:58:14,1,...,TRAVEL,NaN,NaN,NaN,10,24m-222@62j-sjs-v75,42.397326,-71.130785,42.3978092,-71.1303779
3,1.3614527932e+19,42.3743430000,-71.1210580000,42.365028,-71.102423,EAT,SHOP,11:01:04,11:08:23,7,...,TRAVEL,NaN,NaN,NaN,11,zzw-222@62j-sjp-jgk,42.365138,-71.102738,42.3742094,-71.1215814
4,1.8199204595e+19,42.3663241473,-71.1060889361,42.366071,-71.105327,EAT,WORK,11:36:25,11:38:09,1,...,TRAVEL,NaN,NaN,NaN,11,22y-222@62j-sj2-t35,42.365915,-71.105008,42.3665288,-71.1057339


In [797]:
# Save the relevant data for the GAMA simulation model
transportationutilities_trips_df=replica_transportationu_df[['activity_id','start_latitude','start_longitude','end_latitude',
                                                             'end_longitude','travel_purpose','start_time','end_time',
                                                             'duration_minutes','distance_miles','start_hour']]
transportationutilities_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,8.5335040444e+18,42.369628,-71.111400,42.3665288,-71.1057339,EAT,08:52:57,09:00:52,7,0.4099990668,8
1,1.0331853431e+19,42.368530,-71.079323,42.3624471,-71.0800015,EAT,09:37:24,09:40:31,3,0.1616926893,9
2,1.3211394794e+19,42.397326,-71.130785,42.3978092,-71.1303779,EAT,10:57:03,10:58:14,1,0.0617195501,10
3,1.3614527932e+19,42.365138,-71.102738,42.3742094,-71.1215814,EAT,11:01:04,11:08:23,7,1.8652619276,11
4,1.8199204595e+19,42.365915,-71.105008,42.3665288,-71.1057339,EAT,11:36:25,11:38:09,1,0.0901628667,11


## CIVIC INSTITUTIONAL 

In [798]:
# Merge de CIVIC INSTITUTIONAL use buildings: mixed use and civic institutional ones with origin land use 2 = civic institutional
replica_civic_institutional_df=pd.concat([replica_civic_institutional_ci_df,replica_civic_institutional_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_civic_institutional_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,2.8428657676e+18,42.3585045479,-71.1074853134,42.373616,-71.120902,EAT,HOME,05:08:08,05:14:09,6,...,commercial,retail,TRAVEL,NaN,NaN,NaN,5,zzw-222@62j-sjv-975,42.374189,-71.121151
1,5.1115309842e+18,42.3634999995,-71.0969779980,42.363447,-71.097363,EAT,HOME,05:18:06,05:19:30,1,...,commercial,retail,TRAVEL,NaN,NaN,NaN,5,227-222@62j-sj2-ffz,42.363082,-71.096974
2,1.1584559154e+19,42.3641330000,-71.0848400000,42.363621,-71.083286,EAT,MAINTENANCE,05:51:07,05:55:03,3,...,mixed_use,non_retail_attraction,TRAVEL,NaN,NaN,NaN,5,225-223@62j-sj3-3bk,42.364512,-71.082035
3,8.2587979864e+18,42.3649239448,-71.1003423459,42.364362,-71.101517,EAT,HOME,06:08:34,06:11:31,2,...,commercial,retail,TRAVEL,NaN,NaN,NaN,6,227-222@62j-sj2-ffz,42.363082,-71.096974
4,1.8369627728e+19,42.3706046860,-71.1141819628,42.370828,-71.113622,EAT,HOME,06:25:03,06:26:41,1,...,mixed_use,non_retail_attraction,TRAVEL,NaN,NaN,NaN,6,25w-226@62j-sjv-j35,42.369628,-71.111400


In [799]:
# Load the civic institutional OSM data
list_osm_civicinstitutional=['courthouse','embassy','fire_station','police','post_box','post_depot','post_office','prison',
                             'ranger_station','townhall']
osm_civicinstitutional_df=osm_amenities_df[osm_amenities_df['amenity'].isin(list_osm_civicinstitutional)].reset_index().drop('index',axis=1)
osm_civicinstitutional_df.head()

,id,amenity,name,lat,lon
0,257486704,prison,Suffolk County Jail,42.3672757,-71.0663292
1,257493789,police,Cambridge Police Dept,42.3749188,-71.1144991
2,257493900,police,Cambridge Police Department,42.3672754,-71.0861518
3,257493974,police,Harvard University Police,42.3787447,-71.1237790
4,257494413,fire_station,Somerville Fire Department,42.3846647,-71.1113341


In [800]:
# Based on the trip origin lat,lon assign the closest civic institutional buildings with function KDTree
pd.set_option("display.precision", 10)
replica_civic_institutional_df['end_latitude']=''
replica_civic_institutional_df['end_longitude']=''
Z = np.array(osm_civicinstitutional_df[['lat', 'lon']])
tree = spatial.KDTree(Z)
for i in range(len(replica_civic_institutional_df)):
    dist, idxs = tree.query(np.array(replica_civic_institutional_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_civic_institutional_df.at[i, 'end_latitude'] = osm_civicinstitutional_df.at[idxs,'lat']
    replica_civic_institutional_df.at[i, 'end_longitude'] = osm_civicinstitutional_df.at[idxs,'lon']
replica_civic_institutional_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,2.8428657676e+18,42.3585045479,-71.1074853134,42.373616,-71.120902,EAT,HOME,05:08:08,05:14:09,6,...,TRAVEL,NaN,NaN,NaN,5,zzw-222@62j-sjv-975,42.374189,-71.121151,42.3592099,-71.1108903
1,5.1115309842e+18,42.3634999995,-71.0969779980,42.363447,-71.097363,EAT,HOME,05:18:06,05:19:30,1,...,TRAVEL,NaN,NaN,NaN,5,227-222@62j-sj2-ffz,42.363082,-71.096974,42.3648018,-71.0961013
2,1.1584559154e+19,42.3641330000,-71.0848400000,42.363621,-71.083286,EAT,MAINTENANCE,05:51:07,05:55:03,3,...,TRAVEL,NaN,NaN,NaN,5,225-223@62j-sj3-3bk,42.364512,-71.082035,42.3672754,-71.0861518
3,8.2587979864e+18,42.3649239448,-71.1003423459,42.364362,-71.101517,EAT,HOME,06:08:34,06:11:31,2,...,TRAVEL,NaN,NaN,NaN,6,227-222@62j-sj2-ffz,42.363082,-71.096974,42.3637644,-71.1011959
4,1.8369627728e+19,42.3706046860,-71.1141819628,42.370828,-71.113622,EAT,HOME,06:25:03,06:26:41,1,...,TRAVEL,NaN,NaN,NaN,6,25w-226@62j-sjv-j35,42.369628,-71.111400,42.3699518,-71.1131418


In [801]:
# Save the relevant data for the GAMA simulation model
civic_institutional_trips_df=replica_civic_institutional_df[['activity_id','start_latitude','start_longitude','end_latitude',
                                                           'end_longitude','travel_purpose','start_time','end_time',
                                                           'duration_minutes','distance_miles','start_hour']]
civic_institutional_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,2.8428657676e+18,42.374189,-71.121151,42.3592099,-71.1108903,EAT,05:08:08,05:14:09,6,2.6645309436,5
1,5.1115309842e+18,42.363082,-71.096974,42.3648018,-71.0961013,EAT,05:18:06,05:19:30,1,0.0731067069,5
2,1.1584559154e+19,42.364512,-71.082035,42.3672754,-71.0861518,EAT,05:51:07,05:55:03,3,0.2040430223,5
3,8.2587979864e+18,42.363082,-71.096974,42.3637644,-71.1011959,EAT,06:08:34,06:11:31,2,0.1530292708,6
4,1.8369627728e+19,42.369628,-71.111400,42.3699518,-71.1131418,EAT,06:25:03,06:26:41,1,0.0820466950,6


### EDUCATION 

In [802]:
# Merge de EDUCATION use buildings: mixed use, civic institutional, commercial ones with origin land use 2 = education or  with origin land use 2 = retail and previous activity type = SCHOOL
replica_education_df=pd.concat([replica_education_ci_df,retail_education_c_df,replica_education_mu_df,retail_education_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_education_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,1.5976922764e+19,42.3776771912,-71.1137268192,42.372193,-71.115985,EAT,HOME,00:59:15,01:00:58,1,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,0,243-222@62j-sjv-bc5,42.373092,-71.119396
1,1.6915641606e+19,42.3715679261,-71.1204778107,42.371475,-71.116860,EAT,HOME,00:59:15,01:01:21,2,...,commercial,retail,TRAVEL,NaN,NaN,NaN,0,223-222@62j-sjv-9xq,42.370282,-71.118519
2,4.4180069397e+18,42.3728300000,-71.1176230000,42.372895,-71.117836,EAT,SHOP,02:39:50,02:40:17,0,...,commercial,office,TRAVEL,NaN,NaN,NaN,2,227-224@62j-sjv-b6k,42.372484,-71.117852
3,9.7580643697e+18,42.3614780000,-71.0904300000,42.361478,-71.090430,EAT,WORK,03:07:06,03:07:06,0,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,3,222-222@62j-sj4-vcq,42.360213,-71.085994
4,1.3318999804e+19,42.3600920000,-71.0859360000,42.360092,-71.085936,EAT,EAT,04:13:07,04:13:07,0,...,civic_institutional,education,TRAVEL,NaN,NaN,NaN,4,222-222@62j-sj4-vcq,42.360213,-71.085994


In [803]:
# SafeGraph Data for EDUCATION buildings
education_poi_list=['Other Schools and Instruction','Colleges, Universities, and Professional Schools','Technical and Trade Schools',
                    'Administration of Economic Programs','Elementary and Secondary Schools','Child Day Care Services']
education_sg_df=places_df[places_df['top_category'].isin(education_poi_list)]
drop_list=['open_hours','location_name','index']
education_sg_df=education_sg_df.reset_index().drop(drop_list,axis=1)
education_sg_df.head()

,placekey,latitude,longitude,top_category
0,22s-223@62j-sj2-rhq,42.366485,-71.103685,Other Schools and Instruction
1,222-222@62j-sjv-85f,42.373206,-71.118645,"Colleges, Universities, and Professional Schools"
2,226-222@62j-sjv-8vz,42.372416,-71.113445,"Colleges, Universities, and Professional Schools"
3,zzw-224@62j-sj2-p7q,42.359191,-71.093366,Other Schools and Instruction
4,223-223@62j-sj2-vzz,42.371444,-71.081628,Technical and Trade Schools


In [804]:
# Based on the trip origin lat,lon assign the closest EDUCATION building with function KDTree
pd.set_option("display.precision", 8)
replica_education_df['end_latitude']=''
replica_education_df['end_longitude']=''
Z = np.array(education_sg_df[['latitude', 'longitude']])
tree = spatial.KDTree(Z)
for i in range(len(replica_education_df)):
    dist, idxs = tree.query(np.array(replica_education_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_education_df.at[i, 'end_latitude'] = education_sg_df.at[idxs,'latitude']
    replica_education_df.at[i, 'end_longitude'] = education_sg_df.at[idxs,'longitude']
replica_education_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.59769228e+19,42.37767719,-71.11372682,42.372193,-71.115985,EAT,HOME,00:59:15,01:00:58,1,...,TRAVEL,NaN,NaN,NaN,0,243-222@62j-sjv-bc5,42.373092,-71.119396,42.37769,-71.113837
1,1.69156416e+19,42.37156793,-71.12047781,42.371475,-71.116860,EAT,HOME,00:59:15,01:01:21,2,...,TRAVEL,NaN,NaN,NaN,0,223-222@62j-sjv-9xq,42.370282,-71.118519,42.372069,-71.11994
2,4.41800694e+18,42.37283000,-71.11762300,42.372895,-71.117836,EAT,SHOP,02:39:50,02:40:17,0,...,TRAVEL,NaN,NaN,NaN,2,227-224@62j-sjv-b6k,42.372484,-71.117852,42.37298,-71.118463
3,9.75806437e+18,42.36147800,-71.09043000,42.361478,-71.090430,EAT,WORK,03:07:06,03:07:06,0,...,TRAVEL,NaN,NaN,NaN,3,222-222@62j-sj4-vcq,42.360213,-71.085994,42.361889,-71.090546
4,1.33189998e+19,42.36009200,-71.08593600,42.360092,-71.085936,EAT,EAT,04:13:07,04:13:07,0,...,TRAVEL,NaN,NaN,NaN,4,222-222@62j-sj4-vcq,42.360213,-71.085994,42.360623,-71.08743


In [805]:
# Save the relevant data for the GAMA simulation model
education_trips_df=replica_education_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                         'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
education_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.59769228e+19,42.373092,-71.119396,42.37769,-71.113837,EAT,00:59:15,01:00:58,1,0.73943068,0
1,1.69156416e+19,42.370282,-71.118519,42.372069,-71.11994,EAT,00:59:15,01:01:21,2,0.69055975,0
2,4.41800694e+18,42.372484,-71.117852,42.37298,-71.118463,EAT,02:39:50,02:40:17,0,0.02380320,2
3,9.75806437e+18,42.360213,-71.085994,42.361889,-71.090546,EAT,03:07:06,03:07:06,0,0.00000000,3
4,1.33189998e+19,42.360213,-71.085994,42.360623,-71.08743,EAT,04:13:07,04:13:07,0,0.00000000,4


## HEALTHCARE 

In [806]:
# Merge de HEALTHCARE use buildings: mixed use, civic institutional ones with origin land use 2 = healthcare
replica_healthcare_df=pd.concat([replica_healthcare_ci_df,replica_healthcare_mu_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
replica_healthcare_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,destination_land_use_l1,destination_land_use_l2,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude
0,1.56523612e+19,42.37395000,-71.13475800,42.388929,-71.119119,EAT,MAINTENANCE,05:54:12,05:58:31,4,...,commercial,retail,TRAVEL,NaN,NaN,NaN,5,zzw-227@62j-sk2-vs5,42.389415,-71.119372
1,9.28908590e+18,42.39896336,-71.13259449,42.371232,-71.080153,EAT,WORK,07:21:25,07:34:15,12,...,mixed_use,retail,TRAVEL,NaN,NaN,NaN,7,243-222@62j-shx-j7q,42.372246,-71.079820
2,2.32108460e+18,42.37542000,-71.10714100,42.390810,-71.122621,EAT,MAINTENANCE,07:25:46,07:48:42,22,...,commercial,retail,TRAVEL,NaN,NaN,NaN,7,zzw-223@62j-sk2-vs5,42.389362,-71.119548
3,5.74846569e+18,42.39895100,-71.13257300,42.374306,-71.119126,EAT,RECREATION,07:54:33,07:59:54,5,...,commercial,office,TRAVEL,NaN,NaN,NaN,7,zzw-222@62j-sjv-975,42.374189,-71.121151
4,1.72193380e+19,42.39895100,-71.13257300,42.374528,-71.120006,EAT,RECREATION,08:11:59,08:17:55,5,...,commercial,retail,TRAVEL,NaN,NaN,NaN,8,zzw-224@62j-sjv-835,42.374306,-71.119126


In [807]:
# SafeGraph Data for HEALTHCARE buildings
healthcare_poi_list=['Offices of Other Health Practitioners','Offices of Physicians','Personal Care Services','Individual and Family Services',
                    'Outpatient Care Centers','Other Personal Services','Offices of Dentists','Home Health Care Services',
                    'Nursing Care Facilities (Skilled Nursing Facilities)','Medical and Diagnostic Laboratories','General Medical and Surgical Hospitals',
                    'Nursing and Residential Care Facilities','Death Care Services','Specialty (except Psychiatric and Substance Abuse) Hospitals',
                    'Insurance Carriers']
healthcare_sg_df=places_df[places_df['top_category'].isin(healthcare_poi_list)]
drop_list=['open_hours','location_name','index']
healthcare_sg_df=healthcare_sg_df.reset_index().drop(drop_list,axis=1)
healthcare_sg_df.head()

,placekey,latitude,longitude,top_category
0,222-223@62j-shx-qmk,42.364306,-71.079216,Offices of Other Health Practitioners
1,22g-226@62j-sjt-y35,42.374222,-71.134327,Offices of Physicians
2,26d-222@62j-sj2-vs5,42.371577,-71.085370,Personal Care Services
3,zzw-222@62j-sjs-2tv,42.388973,-71.144351,Outpatient Care Centers
4,zzw-226@62j-sjv-9fz,42.373327,-71.123347,Other Personal Services


In [808]:
# Based on the trip origin lat,lon assign the closest HEALTHCARE building with function KDTree
pd.set_option("display.precision", 8)
replica_healthcare_df['end_latitude']=''
replica_healthcare_df['end_longitude']=''
Z = np.array(healthcare_sg_df[['latitude', 'longitude']])
tree = spatial.KDTree(Z)
for i in range(len(replica_healthcare_df)):
    dist, idxs = tree.query(np.array(replica_healthcare_df[['origin_lat', 'origin_lng']].iloc[i]), 1)
    replica_healthcare_df.at[i, 'end_latitude'] = healthcare_sg_df.at[idxs,'latitude']
    replica_healthcare_df.at[i, 'end_longitude'] = healthcare_sg_df.at[idxs,'longitude']
replica_healthcare_df.head()

,activity_id,origin_lat,origin_lng,destination_lat,destination_lng,travel_purpose,previous_activity_type,start_time,end_time,duration_minutes,...,trip_type,vehicle_type,vehicle_fuel_type,vehicle_fuel_technology,start_hour,placekey,start_latitude,start_longitude,end_latitude,end_longitude
0,1.56523612e+19,42.37395000,-71.13475800,42.388929,-71.119119,EAT,MAINTENANCE,05:54:12,05:58:31,4,...,TRAVEL,NaN,NaN,NaN,5,zzw-227@62j-sk2-vs5,42.389415,-71.119372,42.37395,-71.134758
1,9.28908590e+18,42.39896336,-71.13259449,42.371232,-71.080153,EAT,WORK,07:21:25,07:34:15,12,...,TRAVEL,NaN,NaN,NaN,7,243-222@62j-shx-j7q,42.372246,-71.079820,42.398641,-71.133083
2,2.32108460e+18,42.37542000,-71.10714100,42.390810,-71.122621,EAT,MAINTENANCE,07:25:46,07:48:42,22,...,TRAVEL,NaN,NaN,NaN,7,zzw-223@62j-sk2-vs5,42.389362,-71.119548,42.375352,-71.10699
3,5.74846569e+18,42.39895100,-71.13257300,42.374306,-71.119126,EAT,RECREATION,07:54:33,07:59:54,5,...,TRAVEL,NaN,NaN,NaN,7,zzw-222@62j-sjv-975,42.374189,-71.121151,42.398641,-71.133083
4,1.72193380e+19,42.39895100,-71.13257300,42.374528,-71.120006,EAT,RECREATION,08:11:59,08:17:55,5,...,TRAVEL,NaN,NaN,NaN,8,zzw-224@62j-sjv-835,42.374306,-71.119126,42.398641,-71.133083


In [809]:
# Save the relevant data for the GAMA simulation model
healthcare_trips_df=replica_healthcare_df[['activity_id','start_latitude','start_longitude','end_latitude','end_longitude',
                                         'travel_purpose','start_time','end_time','duration_minutes','distance_miles','start_hour']]
healthcare_trips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,1.56523612e+19,42.389415,-71.119372,42.37395,-71.134758,EAT,05:54:12,05:58:31,4,2.22413261,5
1,9.28908590e+18,42.372246,-71.079820,42.398641,-71.133083,EAT,07:21:25,07:34:15,12,5.35109301,7
2,2.32108460e+18,42.389362,-71.119548,42.375352,-71.10699,EAT,07:25:46,07:48:42,22,2.16813895,7
3,5.74846569e+18,42.374189,-71.121151,42.398641,-71.133083,EAT,07:54:33,07:59:54,5,1.99140892,7
4,1.72193380e+19,42.374306,-71.119126,42.398641,-71.133083,EAT,08:11:59,08:17:55,5,2.03569188,8


# MERGE

In [810]:
# Merge all the trips to generate the final eat trips dataset
final_eattrips_df=pd.concat([residential_trips_df,industrial_trips_df,office_trips_df,shopping_trips_df,hotel_trips_df,restaurant_trips_df,work_trips_df,non_retail_trips_df,park_trips_df,transportationutilities_trips_df,civic_institutional_trips_df,education_trips_df,healthcare_trips_df]).sort_values(by=['start_time']).reset_index().drop('index',axis=1)
final_eattrips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour
0,3.71345591e+18,42.368530,-71.079323,42.362934,-71.082109,EAT,00:05:50,00:06:27,0,0.03273589,0
1,1.38419628e+19,42.363664,-71.101451,42.375492,-71.110654,EAT,00:13:11,00:36:32,23,1.20961137,0
2,1.66230097e+19,42.372127,-71.086885,42.388293,-71.123928,EAT,00:21:55,00:28:02,6,2.92798766,0
3,1.62290058e+19,42.372127,-71.086885,42.372127,-71.086885,EAT,00:34:13,00:34:32,0,0.01652482,0
4,1.67153631e+19,42.372127,-71.086885,42.372111,-71.087127,EAT,00:45:13,00:45:32,0,0.01652482,0


In [811]:
len(final_eattrips_df)

10352

# CONNECT TRIPS AND SPEND

In [812]:
# Import the spend data from Safegraph
filepath_Replica_Spend =".././02 Data/Replica/Spend/Cambridge/"
replica_spend_inperson_df = pd.read_csv(filepath_Replica_Spend + 'cambridge_ma_restaurants_and_bars_weekof_01032022_to_07042022_in-person_spend.csv')
replica_spend_inperson_df=replica_spend_inperson_df.rename(columns={'count': 'count_inperson'}, errors='raise')
replica_spend_inperson_df=replica_spend_inperson_df.rename(columns={'change_from_baseline': 'change_from_baseline_inperson'}, errors='raise')
replica_spend_online_df = pd.read_csv(filepath_Replica_Spend + 'cambridge_ma_restaurants_and_bars_weekof_01032022_to_07042022_online_spend.csv')
replica_spend_online_df=replica_spend_online_df.rename(columns={'count': 'count_online'}, errors='raise')
replica_spend_online_df=replica_spend_online_df.rename(columns={'change_from_baseline': 'change_from_baseline_online'}, errors='raise')

In [813]:
# Merge both spend files in the same one
replica_spend_df = replica_spend_inperson_df.merge(replica_spend_online_df[['id','count_online', 'change_from_baseline_online']], how='left', left_on='id', right_on='id')
replica_spend_df.head()

,id,id_label,metric,count_inperson,change_from_baseline_inperson,count_online,change_from_baseline_online
0,Week:202216,"Apr 18, 2022",EstimatedSpendHome,6.72898425e+06,0.16528978,1.19499936e+06,-0.44404872
1,Week:202223,"Jun 6, 2022",EstimatedSpendHome,7.48801524e+06,0.23578717,8.26769861e+05,-0.63343902
2,Week:202213,"Mar 28, 2022",EstimatedSpendHome,5.54881832e+06,0.14985503,1.07312424e+06,-0.40258348
3,Week:202203,"Jan 17, 2022",EstimatedSpendHome,5.51725017e+06,0.11191819,1.29166361e+06,-0.30066668
4,Week:202201,"Jan 3, 2022",EstimatedSpendHome,4.12540348e+06,0.00000000,1.53561305e+06,0.00000000


In [814]:
# Calculate percentage of the money spend online comparing to the total
replica_spend_df['%online']=replica_spend_df['count_online']/(replica_spend_df['count_online']+replica_spend_df['count_inperson'])*100
replica_spend_df.head()

,id,id_label,metric,count_inperson,change_from_baseline_inperson,count_online,change_from_baseline_online,%online
0,Week:202216,"Apr 18, 2022",EstimatedSpendHome,6.72898425e+06,0.16528978,1.19499936e+06,-0.44404872,15.08079037
1,Week:202223,"Jun 6, 2022",EstimatedSpendHome,7.48801524e+06,0.23578717,8.26769861e+05,-0.63343902,9.94337017
2,Week:202213,"Mar 28, 2022",EstimatedSpendHome,5.54881832e+06,0.14985503,1.07312424e+06,-0.40258348,16.20558066
3,Week:202203,"Jan 17, 2022",EstimatedSpendHome,5.51725017e+06,0.11191819,1.29166361e+06,-0.30066668,18.97018609
4,Week:202201,"Jan 3, 2022",EstimatedSpendHome,4.12540348e+06,0.00000000,1.53561305e+06,0.00000000,27.12610080


In [815]:
# Calculate de average percentage of online spend
average=replica_spend_df['%online'].mean()
average

16.458436887945336

In [816]:
# Definition of inperson vs first iteration of online trips
in_person_trips=len(eattrips_df)+l_nothing
online_trips=in_person_trips*average/(100-average)

In [817]:
# Comparison of the trips needed to do online vs in person depending on the order size
avg_online_order=33.26
avg_inperson_order=60.40
travels=avg_inperson_order/avg_online_order
# Calculation of real online trips
online_trips=round(online_trips*travels)
# Calculation of the trips to delate
delete_trips=in_person_trips-online_trips

In [818]:
delete_trips

7165

In [819]:
final_eattrips_df['same_place']=' '
count_same = 0
for i in range(len(final_eattrips_df)):
    if (final_eattrips_df.at[i,'start_latitude']==final_eattrips_df.at[i,'end_latitude'])&(final_eattrips_df.at[i,'start_longitude']==final_eattrips_df.at[i,'end_longitude']):
        final_eattrips_df.at[i,'same_place']='Yes'
        count_same = count_same + 1
    else:
        final_eattrips_df.at[i,'same_place']='No'

final_eattrips_df=final_eattrips_df[final_eattrips_df['same_place']=='No']
final_eattrips_df=final_eattrips_df.reset_index().drop('index',axis=1)
final_eattrips_df.head()

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour,same_place
0,3.71345591e+18,42.368530,-71.079323,42.362934,-71.082109,EAT,00:05:50,00:06:27,0,0.03273589,0,No
1,1.38419628e+19,42.363664,-71.101451,42.375492,-71.110654,EAT,00:13:11,00:36:32,23,1.20961137,0,No
2,1.66230097e+19,42.372127,-71.086885,42.388293,-71.123928,EAT,00:21:55,00:28:02,6,2.92798766,0,No
3,1.67153631e+19,42.372127,-71.086885,42.372111,-71.087127,EAT,00:45:13,00:45:32,0,0.01652482,0,No
4,2.74441125e+18,42.362611,-71.098646,42.36447,-71.100846,EAT,00:47:20,00:52:15,4,0.25483028,0,No


In [820]:
final_eattrips_df_copy=final_eattrips_df.copy()
final_eattrips_df=final_eattrips_df[(final_eattrips_df['start_time']=='16:04:58')==False]
count_distance_zero=final_eattrips_df[final_eattrips_df['duration_minutes']==0].shape[0]
final_eattrips_df=final_eattrips_df[(final_eattrips_df['duration_minutes']==0)==False]
final_eattrips_df=final_eattrips_df.reset_index().drop('index',axis=1)
len(final_eattrips_df)

8862

In [821]:
# Same of the trips that need to be deleted have already been deleted in steps before
extra_delete_trips=delete_trips-l_nothing-count_same-count_distance_zero-1

In [823]:
# Delete random trips and generate the online delivery dataset
from numpy.random import default_rng

trips_drop = default_rng().choice(final_eattrips_df.index, size=extra_delete_trips, replace=False)
onlinetrips_df=final_eattrips_df.drop(index=trips_drop).reset_index().drop('index',axis=1)

In [824]:
# Change format of start_time and end_time to be able to read in GAMA
onlinetrips_df['start_time']=pd.to_datetime(onlinetrips_df['start_time'])
onlinetrips_df['end_time']=pd.to_datetime(onlinetrips_df['end_time'])
onlinetrips_df

,activity_id,start_latitude,start_longitude,end_latitude,end_longitude,travel_purpose,start_time,end_time,duration_minutes,distance_miles,start_hour,same_place
0,1.66230097e+19,42.372127,-71.086885,42.388293,-71.123928,EAT,2022-10-11 00:21:55,2022-10-11 00:28:02,6,2.92798766,0,No
1,1.69156416e+19,42.370282,-71.118519,42.372069,-71.11994,EAT,2022-10-11 00:59:15,2022-10-11 01:01:21,2,0.69055975,0,No
2,1.59769228e+19,42.373092,-71.119396,42.37769,-71.113837,EAT,2022-10-11 00:59:15,2022-10-11 01:00:58,1,0.73943068,0,No
3,2.43972238e+18,42.369111,-71.109872,42.372419,-71.116313,EAT,2022-10-11 03:29:57,2022-10-11 03:31:27,1,0.58990422,3,No
4,1.48233700e+19,42.389247,-71.120337,42.382257,-71.123093,EAT,2022-10-11 04:28:33,2022-10-11 04:40:53,12,0.63936869,4,No
...,...,...,...,...,...,...,...,...,...,...,...,...
3984,7.19437858e+18,42.372746,-71.099375,42.373422,-71.089086,EAT,2022-10-11 23:05:32,2022-10-11 23:21:46,16,0.84144375,23,No
3985,1.82569034e+19,42.362782,-71.093880,42.362183,-71.100137,EAT,2022-10-11 23:12:35,2022-10-11 23:16:30,3,0.20365808,23,No
3986,1.12700286e+18,42.383291,-71.119836,42.372069,-71.11994,EAT,2022-10-11 23:13:44,2022-10-11 23:31:27,17,0.91764851,23,No
3987,8.44889048e+18,42.383291,-71.119836,42.369952,-71.113142,EAT,2022-10-11 23:13:44,2022-10-11 23:37:23,23,1.22239321,23,No


In [825]:
# Save the results of final food delivery trips
results_path=Path('./Input Database/fooddeliverytrips_cambridge.csv')
onlinetrips_df.to_csv(results_path,index=False)